In [1]:
import pandas as pd
from wordcloud import WordCloud
import datetime as dt
import re

import emoji

from collections import defaultdict

import nltk
from nltk.corpus import stopwords

import plotly.graph_objs as go

from io import BytesIO
import base64

from dash.exceptions import PreventUpdate
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

In [2]:
def create_df(df_str):
    dict_replace = {"Ravindra Bhandarkar🐼" : "Ravindra" , 'Nagraj Infy':"Nagaraj", 'Sughosh Sp Nitte':"Sughosh",\
         'Nishant Shenoy Nitte':"Nishant", 'Bhuvan Reddy' : "Bhuvan", 'Raymond Pinto':"Raymond", 'Kavan Mandappa':"Mandappa",\
        'Sharath New':"Sharath", 'Adarsha Viper':"Adarsh", 'Manoj Gaonkar':"Manoj", 'Jeevan M H':"Jeevan", 'H J Bhargav':"Bhargav",\
        'Saiprakash Nitte' : "Sai Prakash", 'Shrinay Nitte':"Shrinay", 'Prasanna Hostel':"Prasanna", 'Prajwal Tantry':"Tantry",\
         'Pradeep Section':"Pradeep", 'Maneesh Dali Vada':"Maneesh", 'Shravan Star Player':"Shravan", 'Jason Spice' : "Jason",\
         'Datta Sec':"Datta", 'Vaishnav Infy':"Vaishnav", 'Shekhar nitte':"Shekhar", ". n Sai Prakash":"Sai Prakash",\
        ". n Bhuvan":"Bhuvan", ". n Shrinay" :"Shrinay", ". n Shravan":"Shravan",". n Raymond":"Raymond",". n Bharghav":"Bhargav",\
         ". n Mandappa":"Mandappa", ". n Tantry":"Tantry",". n Adarsh":"Adarsh",". n Sharath":"Sharath",". n Datta":"Datta",\
          ". n Sughosh":"Sughosh",". n Jason":"Jason", ". n Prasanna":"Prasanna",". n Maneesh":"Maneesh", ". i Nagaraj":"Nagaraj",\
          "Mazeen Muhammed":"Mazeen",". i Vaishnav":"Vaishnav", ". n Shekhar":"Shekhar",". n Jeevan":"Jeevan",\
         ". n Pradeep":"Pradeep",". n Nishant":"Nishant",". n Ravindra":"Ravindra",". n Manoj":"Manoj"}

    parsedData = []

    iphone_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*): "
    iphone_left_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*) left$"
    iphone_subject_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%-*{}\"';><,/?`~]*)changed the subject to ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^:()&%\-*{}\"';><,/?`~]*)"
    iphone_icon_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%-*{}\"';><,/?`~]*) (changed this group's icon)$"
    iphone_added_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%-*{}\"';><,/?`~]*)added([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^:()&%\-*{}\"';><,/?`~]*)"
    iphone_removed_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%-*{}\"';><,/?`~]*)removed([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^:()&%\-*{}\"';><,/?`~]*)"
    iphone_anything_else_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) "

    pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*): "
    left_pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*)left$"
    left = []
    icon_pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*)(changed this group's icon)$"
    icon = []
    subject_pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%-*{}\"';><,/?`~]*)changed the subject from ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^:()&%\-*{}\"';><,/?`~]*) to ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^:()&%\-*{}\"';><,/?`~]*)"
    subject = []
    added_pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*) added ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*)"
    added = []
    removed_pattern  = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*) removed ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*)"
    removed = []
    anything_else_pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - "
    anything_else = []
    diff = []
    
    data = df_str.split('\n')
    if data[0].replace('\u200e','').strip('\n').startswith('['):

        for x in data[-len(data)+3:]:
            x = x.replace('\u200e','').strip('\n').strip('\r')
            if re.match(iphone_left_pattern,x):
                name = x.split(']')[1].strip("left").strip()
                left.append(name)
            elif re.match(iphone_subject_pattern,x):
                name = x.split("changed the subject to")[0].split(']')[1].strip().replace('You','Ravindra Bhandarkar🐼')
                subject.append(name)
            elif re.match(iphone_icon_pattern,x):
                name = x.split("changed this group's icon")[0].split(']')[1].strip().replace('You','Ravindra Bhandarkar🐼')
                icon.append(name)
            elif re.match(iphone_added_pattern,x):
                added.append(x)
            elif re.match(iphone_removed_pattern,x):
                removed.append(x)
            elif re.match(iphone_pattern,x):
                parsedData.append(x)
            elif not re.match(iphone_anything_else_pattern,x):
                parsedData[-1] = parsedData[-1]+' '+x
            else:
                diff.append(x)
        final_list= []    
        for i in parsedData:
            split = i.strip().split("] ",1)
            Date = split[0].strip('[')
            Author, Message = split[1].split(':',1)
            final_list.append([dt.datetime.strptime(Date.replace(",",""), "%d/%m/%y %I:%M:%S %p"),Author.strip(), Message.strip()]) 

    else:
        for x in data[-len(data)+3:]:
            x = x.replace('\u200e','').strip('\n')
            if re.match(left_pattern,x):
                name = x.split('-')[1].strip("left").strip()
                left.append(name)
            elif re.match(subject_pattern,x):
                name = x.split("changed the subject from")[0].split('-')[1].strip().replace('You','Mazeen Muhammed')
                subject.append(name)
            elif re.match(icon_pattern,x):
                name = x.split("changed this group's icon")[0].split('-')[1].strip().replace('You','Mazeen Muhammed')
                icon.append(name)
            elif re.match(added_pattern,x):
                added.append(x)
            elif re.match(removed_pattern,x):
                removed.append(x)
            elif re.match(pattern,x):
                parsedData.append(x)
            elif not re.match(anything_else_pattern,x):
                parsedData[-1] = parsedData[-1]+' '+x
            else:
                diff.append(x)
        final_list= []    
        for i in parsedData:
            Date,auth_msg = i.strip().split(' - ',1)
            Author, Message = auth_msg.split(":",1)
            final_list.append([dt.datetime.strptime(Date.replace(",",""), "%d/%m/%y %I:%M %p"),Author, Message]) 

    data_frame=pd.DataFrame(final_list , columns=["Date","Author","Message"]).dropna()
    data_frame['Year'] = data_frame.Date.map(lambda x: x.year)
    data_frame['Month'] = data_frame.Date.map(lambda x: x.month)
    data_frame['Month_Name'] = data_frame.Date.map(lambda x: x.month_name())
    data_frame['Day'] = data_frame.Date.map(lambda x: x.day)
    data_frame['Day_Number'] = data_frame.Date.map(lambda x: x.isoweekday())
    data_frame['Day_Of_Week'] = data_frame.Date.map(lambda x: x.day_name())
    data_frame['Hour'] = data_frame.Date.map(lambda x: x.hour)
    data_frame['Mins'] = data_frame.Date.map(lambda x: x.minute)
    data_frame.Author.replace(dict_replace, inplace=True) ## remove this later
    data_frame.Message = data_frame.Message.map(lambda x : '<Media omitted>' if  x.endswith('omitted') else x)
    data_frame.dropna(inplace=True, axis=0)
    
    df_exits = pd.DataFrame()
    if(left):
        df_exits=pd.DataFrame(pd.DataFrame(left).replace(dict_replace).value_counts()).rename(columns={0:'Left'}).reset_index().rename(columns={0:'Authors'})
            
    df_icon = pd.DataFrame()
    if(icon):
        df_icon=pd.DataFrame(pd.DataFrame(icon).replace(dict_replace).value_counts()).rename(columns={0:'Icon'}).reset_index().rename(columns={0:'Authors'})
    
    df_subject = pd.DataFrame()
    if(subject):
        df_subject=pd.DataFrame(pd.DataFrame(subject).replace(dict_replace).value_counts()).rename(columns={0:'Subject'}).reset_index().rename(columns={0:'Authors'})

    return data_frame, df_exits, df_icon, df_subject

In [3]:
def create_sub_data_frames(data_frame):
    df_with_media = data_frame[['Date','Author', 'Message']].copy()
    df_with_media['Message'] = df_with_media.Message.map(lambda x : "Media is present" if ("<Media omitted>" in x) else "No Media")
    df_with_media = df_with_media[df_with_media.Message=='Media is present'].reset_index(drop=True)
    df_with_media.dropna(inplace =True)

    df_without_media = data_frame[['Date','Author', 'Message']].copy()
    df_without_media['Message'] = df_without_media.Message.map(lambda x : x if "<Media omitted>" not in x else None)
    df_without_media = df_without_media.dropna().reset_index(drop=True)

    df_without_media_link = df_without_media.copy()
    df_without_media_link['new_col'] = df_without_media_link.Message.map(lambda x : x if not re.findall(r'(https?://\S+)', x) else None)
    df_without_media_link = df_without_media_link.dropna().reset_index(drop=True).iloc[:,:-1]
    remove_emoji_pattern = re.compile(pattern = "[\u200d♂♥️\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639\W\d]+", flags = re.UNICODE)
    df_without_media_link_emoji = df_without_media_link.copy()
    df_without_media_link_emoji['Message'] = df_without_media_link_emoji.Message.map(lambda x : remove_emoji_pattern.sub(r' ',x) if len(remove_emoji_pattern.sub(r'',x).strip())>0 else None).dropna()
    df_without_media_link_emoji.dropna(inplace = True)
    df_without_media_link_emoji.reset_index(inplace=True)

    return df_with_media, df_without_media, df_without_media_link_emoji

In [4]:
def create_data_frames_by_category(data_frame, df_with_media, df_without_media, df_without_media_link_emoji):
    # creating links data frame
    ls_authors = df_without_media.Author.unique().tolist()
    dic_links_authors_df = {}
    for author in ls_authors:
        ls_msg = df_without_media[['Author','Message']][df_without_media.Author==author].Message.tolist()
        ls_link = []
        for msg in ls_msg:
            links = re.findall(r'(https?://\S+)', msg)
            for link in links:
                ls_link.append(re.findall ("^https?://([\w.]*)/",link))
        if len(ls_link)>0:
            link_df = pd.DataFrame(ls_link).replace({"youtu.be":"www.youtube.com", "instagram.com":"www.instagram.com"}).rename(columns={0:'Links'})
            link_df = pd.DataFrame(link_df.groupby("Links").Links.count()).rename(columns = {'Links':'Total_Count'})\
            .sort_values("Total_Count", ascending=False).reset_index().rename(columns = {"Links":"Links sent by "+author})
            dic_links_authors_df[author] = link_df
    if len(dic_links_authors_df.keys()):
        ls_links_author = []
        ls_total_count = []
        for i  in dic_links_authors_df:
            ls_links_author.append(i)
            ls_total_count.append(dic_links_authors_df[i].Total_Count.sum())
        links_data_frame = pd.DataFrame([ls_links_author,ls_total_count]).T.rename(columns = {0:'Author', 1: 'Links'}).set_index('Author').sort_index()
        links_data_frame = links_data_frame.sort_values('Links',ascending=False)

        ls_msg = df_without_media.Message.tolist()
        ls_link = []
        for msg in ls_msg:
            links = re.findall(r'(https?://\S+)', msg)
            for link in links:
                ls_link.append(re.findall ("^https?://([\w.]*)/",link))
        link_df = pd.DataFrame(ls_link).replace({"youtu.be":"www.youtube.com", "instagram.com":"www.instagram.com"}).rename(columns={0:'Overall links sent in group'})
        overall_link_df = pd.DataFrame(link_df.groupby("Overall links sent in group")['Overall links sent in group'].count()).rename(columns = {'Overall links sent in group':'Total_Count'}).sort_values("Total_Count", ascending=False)
        dic_links_authors_df['Overall'] = overall_link_df
    else:
        links_data_frame = pd.DataFrame([ls_authors,[0]*len(ls_authors)]).T.rename(columns = {0:'Author', 1: 'Links'}).set_index('Author').sort_index()
        links_data_frame = links_data_frame.sort_values('Links',ascending=False)

    # creating words data frame
    stopset = set(stopwords.words('english')) 
    stopset.update({"","yeah","yes","okay","like","get","still","take","get","let","u","oh","n","sa","also","make"})

    string = ' '.join(df_without_media_link_emoji.Message)
    emoji_special_pattern = '([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \W\d])'
    string_without_emo_spec_dig = re.sub(emoji_special_pattern, " ",string)
    dic_df_author_word_count = {}
    dic_author_total_and_unique_words =defaultdict(list)
    for author in ls_authors:
        string_without_emo_spec_dig = ' '.join(df_without_media_link_emoji[['Author','Message']][df_without_media_link_emoji.Author==author].Message)

        default_dic_author_selected = defaultdict(int)
        ls_word_author = string_without_emo_spec_dig.lower().split()
        for word in ls_word_author:
            if (word.strip() not in stopset):
                default_dic_author_selected[word.strip()]+=1
        author_word_df = pd.DataFrame([default_dic_author_selected.keys(),default_dic_author_selected.values()]).T.rename(columns = {0 : 'Word_For_'+author,1:'Total_Count'}).sort_values(['Total_Count'], ascending=False)
        dic_df_author_word_count[author] = author_word_df 
        default_dic_author_wordcount = defaultdict(int)
        
        for i in ls_word_author:
            default_dic_author_wordcount[i]+=1
        dic_author_total_and_unique_words[author] = [len(default_dic_author_wordcount), len(ls_word_author)]
    ls_words_author = []
    ls_total_count = []
    ls_unique_count = []
    for i in dic_author_total_and_unique_words:
        ls_words_author.append(i)
        ls_total_count.append(dic_author_total_and_unique_words[i][1])
        ls_unique_count.append(dic_author_total_and_unique_words[i][0])
    words_data_frame = pd.DataFrame([ls_words_author,ls_unique_count , ls_total_count]).T.rename(columns = {0:'Author', 1: 'Unique_Words', 2:'Words'}).set_index('Author').sort_index()
    words_data_frame = words_data_frame.sort_values('Words',ascending=False)
    
    string = ' '.join(df_without_media_link_emoji.Message)
    emoji_special_pattern = '([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \W\d])'
    string_without_emo_spec_dig = re.sub(emoji_special_pattern, " ",string)


    default_dic_overall = defaultdict(int)
    ls_word = string_without_emo_spec_dig.lower().split()
    for i in ls_word:
        if i not in stopset:
            default_dic_overall[i]+=1
    overall_word_df = pd.DataFrame([default_dic_overall.keys(),default_dic_overall.values()]).T.rename(columns = {0 : 'Words',1:'Total_Count'}).sort_values(['Total_Count'], ascending=False).set_index('Words')
    dic_df_author_word_count['Overall'] = overall_word_df
    
    ## Create Emojis Data Frame

    dic_author_str_without_media = {} # dictionary of message string per person
    dic_author_default_emoji = {}     # dictionary default dic 
    dic_author_emoji_df = {}          # dictionary of data frames per emoji author 
    for author in ls_authors:
        str_without_media = ' '.join(df_without_media[['Author','Message']][df_without_media.Author==author].Message)
        dic_author_str_without_media[author] = str_without_media

        dic_author_emoji = [emo['emoji'] for emo in emoji.emoji_lis(str_without_media)]
        default_emoji = defaultdict(int)
        for emo in dic_author_emoji:
            default_emoji[emo]+=1

        dic_author_default_emoji[author] = dict(default_emoji)
        emoji_df = pd.DataFrame([dic_author_default_emoji[author].keys(), dic_author_default_emoji[author].values()]).T\
                    .rename(columns = {0:'Emojis used by '+author, 1:"Total_Count"}).sort_values('Total_Count', ascending=False).set_index('Emojis used by '+author)
        emoji_df['Name'] = emoji_df.index.map(lambda x : ' '.join(emoji.demojize(x).strip(":").split("_"))).tolist()
        emoji_df['Name'] = emoji_df.Name.map(lambda x :x if not(x.endswith('tone')  or x.startswith('regional') or \
                                      ((x.startswith('male') or x.startswith('female'))  \
                                       and x.endswith('sign'))) else None)#.dropna()
        emoji_df.dropna(inplace=True)
        if(len(emoji_df)!=0):
            dic_author_emoji_df[author] = emoji_df

    ls_emoji_author = []
    ls_total_count = []
    for i  in dic_author_emoji_df:
        ls_emoji_author.append(i)
        ls_total_count.append(dic_author_emoji_df[i].Total_Count.sum())
    emojis_data_frame = pd.DataFrame([ls_emoji_author,ls_total_count]).T.rename(columns = {0:'Author', 1: 'Emojis'}).set_index('Author').sort_index()
    emojis_data_frame = emojis_data_frame.sort_values('Emojis',ascending=False)
    
    str_without_media = ' '.join(df_without_media.Message)
    emoji_list = [i['emoji'] for i in emoji.emoji_lis(str_without_media)]

    default_emoji = defaultdict(int)
    for i in emoji_list:
        default_emoji[i]+=1
    overall_emoji_df = pd.DataFrame([default_emoji.keys(),default_emoji.values()]).T.rename(columns = {0 : 'Emoji',1:'Total_Count'}).sort_values(['Total_Count'], ascending=False).set_index('Emoji')
    overall_emoji_df['Name'] = overall_emoji_df.index.map(lambda x : ' '.join(emoji.demojize(x).strip(":").split("_"))).tolist()

    overall_emoji_df['Name'] = overall_emoji_df.Name.map(lambda x :x if not(x.endswith('tone')  or x.startswith('regional') or \
                                      ((x.startswith('male') or x.startswith('female'))  \
                                       and x.endswith('sign'))) else None)#.dropna()
    overall_emoji_df.dropna(inplace=True)
    dic_author_emoji_df['Overall'] = overall_emoji_df

    ## Create Media data frame
    medias_data_frame = df_with_media.groupby('Author').count().sort_index().iloc[:,1:2].rename(columns = {'Message':"Medias"})
    medias_data_frame = medias_data_frame.sort_values('Medias', ascending = False)

    ## Create Messages data frame
    messages_data_frame = data_frame.groupby('Author').count().iloc[:,1:2].rename(columns = {'Message':"Messages"})
    messages_data_frame = messages_data_frame.sort_values('Messages', ascending = False)

    chat_analysis_data_frame = pd.concat([links_data_frame,words_data_frame,emojis_data_frame,medias_data_frame,messages_data_frame], axis=1).fillna(0)
    chat_analysis_data_frame.Links = chat_analysis_data_frame.Links.astype(int)
    chat_analysis_data_frame.Unique_Words = chat_analysis_data_frame.Unique_Words.astype(int)
    chat_analysis_data_frame.Words = chat_analysis_data_frame.Words.astype(int)
    chat_analysis_data_frame.Emojis = chat_analysis_data_frame.Emojis.astype(int)
    chat_analysis_data_frame.Medias = chat_analysis_data_frame.Medias.astype(int)
    chat_analysis_data_frame.Messages = chat_analysis_data_frame.Messages.astype(int)
    chat_analysis_data_frame.reset_index(inplace= True)
    chat_analysis_data_frame.columns = ['Group Member' , 'Links',  'Unique_Words' , 'Words',  'Emojis',  'Medias',  'Messages']
    return chat_analysis_data_frame.sort_values(['Messages' , 'Words' , 'Unique_Words' , 'Medias' , 'Emojis', 'Links'], ascending = False )[['Group Member','Messages' , 'Words' , 'Unique_Words' , 'Medias' , 'Emojis', 'Links']] , dic_links_authors_df, dic_author_emoji_df, dic_df_author_word_count

In [5]:
def get_chart_basic_info(data_frame):
    progress = data_frame.copy()
    ls_years = list((progress.groupby('Year').groups.keys()))
    ls_years.append('All')
    
    top_authors = progress.groupby('Author').count().sort_values(['Message'], ascending=False).Message.index

    monthly_progress= pd.DataFrame()

    ls_years_df = []
    for i in ls_years[:-1]:
        monthly_progress  = progress[progress.Year==i]
        ls_years_df.append(monthly_progress)

    ls_years_df.append(progress)

    axis_style = dict(
            fixedrange = True,
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(family='Arial',size=12,color='rgb(82, 82, 82)',),
    )

    hover_label_style = dict( bgcolor="white",font_size=15)
    margin_style = dict(l=100, r=20, t=100,)
    button_style = dict( active=0, direction="down", pad={"r": 0, "t": 10}, x=0.03, xanchor="left", y=1.1, yanchor="top")
    
    return ls_years, top_authors, ls_years_df, axis_style, hover_label_style, margin_style, button_style

In [6]:
def fig_overall_messages(hover_label_style, axis_style, margin_style, button_style, ls_years_df, ls_years  ) :
    fig = go.Figure()
    OverAll_month_series = ls_years_df[-1][['Month','Month_Name','Message']].groupby(['Month','Month_Name']).count()
    ls_data = []
    counter = 0
    for year_df in ls_years_df[:-1]:
        dict_month = {'January':0,'February':0,'March':0,'April':0, 'May':0,'June':0,'July':0,'August':0,'September':0,'October':0,'November':0,'December':0}    
        year_df = year_df[['Month','Month_Name','Message']].groupby(['Month','Month_Name']).count()
        index = year_df.index.get_level_values(1).tolist()
        value = year_df.Message.tolist()
        for i in range(len(index)):
            dict_month[index[i]] = value[i]

        ls_data.append(go.Scatter( x = list(dict_month.keys()), y = list(dict_month.values()),
              mode = 'lines+markers', marker =dict(size=1, line={'width':2 }), name  = ls_years[counter], cliponaxis = False ))
        counter+=1

    dict_month = {'January':0,'February':0,'March':0,'April':0, 'May':0,'June':0,'July':0,'August':0,'September':0,'October':0,'November':0,'December':0}    

    index = OverAll_month_series.index.get_level_values(1).tolist()
    value = OverAll_month_series.Message.tolist()
    for i in range(len(index)):
        dict_month[index[i]] = value[i]

    ls_data.append(go.Bar( x = list(dict_month.keys()), y = list(dict_month.values()), opacity =0.3,
          name  = "Overall", text=list(dict_month.values()), textposition='outside', hoverinfo = 'skip'
                ))

    fig = go.Figure( data = ls_data )

    ls_buttons = [dict(label="Show All Plots",
                         method="update",
                         args=[{"visible": True},
                               {"title": "Total group messages per month"},
                              ])]
    ls_true_false = [False]*len(ls_years)
    for i in range(len(ls_years)):
        ls_true_false[i] = True
        ls_buttons.append(dict(label=ls_years[i],
                         method="update",
                         args=[{"visible": ls_true_false},
                               {"title": "Total group messages per month for the year {}".format(ls_years[i])}]))
        ls_true_false = [False]*len(ls_years)

    fig.update_layout( 
        height = 500,
        title_text = "Total group messages per month",
        hoverlabel=hover_label_style,
        hovermode = 'x',
        xaxis_title="Months",
        yaxis_title="Total number of messages",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        showlegend=False,
        plot_bgcolor='white',
        updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )
    return fig

In [7]:
def fig_messages_per_member_continuous(progress, hover_label_style, axis_style, margin_style, button_style, top_authors ) :
    ls_data = []
    fig = go.Figure()
    for top in top_authors:
        author_df = progress[['Author','Date','Message']][progress.Author==top].set_index('Date').resample('m').count()
        ls_data.append(go.Scatter(x =author_df.index, y =author_df.Message,
              mode = 'lines+markers', marker =dict(size=1, line={'width':2 }), name  =top))

    fig = go.Figure( data = ls_data )
    ls_buttons=[]
    ls_buttons = [dict(label= "Show All",
                         method="update",
                         args=[{"visible": True},
                               {"title": "Continuous monthly progress"},
                              ])]

    ls_true_false = [False]*len(top_authors)
    for i in range(len(top_authors)):
        ls_true_false[i] = True

        ls_buttons.append(dict(label="Top {}".format(i+1),
                         method="update",
                         args=[{"visible": list(ls_true_false)},
                               {"title": "Continuous monthly progress for Top {}".format(i+1)}]))
    fig.update_layout(
        height = 500,
        title_text = 'Continuous monthly progress of group members',
        hovermode="x",
        hoverlabel=hover_label_style,
        xaxis_title="Months",
        yaxis_title="Total number of messages",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        showlegend=True,
        plot_bgcolor='white',
        updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )
    return fig

In [8]:
def fig_bar_chart_mess_per_person(progress, selected_year, hover_label_style, axis_style, margin_style ) :
    fig = go.Figure()
    TotalMessages = progress.groupby('Author').count().iloc[:,1:2].rename(columns = {'Message':"Total_Messages"}).sort_values('Total_Messages', ascending = False)
    TotalMessages['Percentage_Total'] = round((TotalMessages.Total_Messages/TotalMessages.Total_Messages.sum()*100),2)
    data = [
        go.Bar(x = TotalMessages.index , y= TotalMessages.Total_Messages , name='', customdata=TotalMessages,
               hovertemplate = '%{x}<br>'+ 
               'Total Messages = %{y}<br>'+
               'Percentage of Total Messages = %{customdata[1]}%',
               text = TotalMessages.Total_Messages , textposition = 'outside', cliponaxis = False
              )
    ]

    fig = go.Figure(data= data)

    fig.update_layout( 
        hoverlabel = hover_label_style,
        title_text='Total number of messages per person',
        xaxis_title="Members",
        yaxis_title="Total number of messages",
        xaxis = axis_style,
        yaxis = axis_style,
        margin = margin_style,
        plot_bgcolor='white',
    )
    ls_colours = []
    for i in range(len(TotalMessages)):
        ls_colours.append("rgb"+str((51+i*6, 190, 57+i*6)))

    fig.update_traces(
     marker={"color": ls_colours, }, 
    )
    fig.update_traces(
 marker={"color": "#2dc272", }, opacity = 0.8
)
    return fig

In [9]:
def fig_line_chart_mess_per_person_based_year(progress, selected_year, hover_label_style, axis_style, margin_style, button_style) :
    fig = go.Figure()
    ls_data = []
    top_authors = progress.groupby('Author').count().sort_values(['Message'], ascending=False).Message.index
    for top in top_authors:
        author_df = progress[["Author","Message","Month","Month_Name"]][progress.Author==top].groupby(["Month","Month_Name"]).count()
        ls_data.append(go.Scatter(x =author_df.index.get_level_values(1).tolist(), y =author_df.Message.tolist(),
              mode = 'lines+markers', marker =dict(size=1, line={'width':2 }), name  =top))

    fig = go.Figure( data = ls_data )

    ls_buttons=[]
    ls_buttons = [dict(label= "Show All",
                         method="update",
                         args=[{"visible": True},
                               {"title": 'Total number of messages for all group members'},
                              ])]
    ls_true_false = [False]*len(top_authors)
    for i in range(len(top_authors)):
        ls_true_false[i] = True
        ls_buttons.append(dict(label="Top {}".format(i+1),
                         method="update",
                         args=[{"visible": list(ls_true_false)},
                               {"title": "Total number of messages for Top {}".format(i+1)}]))

    fig.update_layout(
        title_text = 'Total number of messages for all group members',
        hovermode="x unified",
        hoverlabel=hover_label_style,
        xaxis_title="Months",
        yaxis_title="Total number of messages",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        showlegend=True,
        plot_bgcolor='white',
            updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )
    return fig

In [10]:
def fig_active_day_hour_date(progress, selected_year, hover_label_style, axis_style, margin_style):
    days = progress[['Day_Number','Day_Of_Week','Message']].groupby(['Day_Number','Day_Of_Week']).count()
    data_day = [go.Bar(x = days.index.get_level_values(1).tolist(), y = days.Message.tolist(), name='selected_year',
                text=['{:,}'.format(i) for i in days.Message.tolist()],textposition='outside', cliponaxis = False
                      )]

    fig_day = go.Figure(data = data_day)

    fig_day.update_layout( 
        title_text='Active days of the week',
        xaxis_title="Days of the week",
        yaxis_title="Total number of messages",
        hoverlabel=hover_label_style,
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
        showlegend=False
    )

    hour = progress[['Hour','Message']].groupby(['Hour']).count()

    data_hour = [
        go.Bar(x = hour.index.tolist() , y = hour.Message.tolist() , name='',
           hovertemplate = 'Hour of the day = %{x}<br>'+ 
           'Total Messages = %{y}<br>',
           text = ['{:,}'.format(i) for i in hour.Message.tolist()] , textposition = 'outside', cliponaxis = False
          )]

    fig_hour = go.Figure(data = data_hour)
    fig_hour.update_layout( 
        hoverlabel = dict(bgcolor ='white', bordercolor='grey'),
        title_text='Active hours of the day',
        xaxis_title="Hours 12:00 A.M to 11:00 PM",
        yaxis_title="Total Number of messages",
        xaxis = {**{'tick0': 0,'dtick': 1},**axis_style}, # this tick0 sys show the starting tick and dtick says next tick,
                                                        # if dtick was 2, then it wouold show alternate ticks
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
    )
    date = progress[['Day','Message']].groupby('Day').count()

    data_date = [
    go.Bar(x = date.index.tolist() , y = date.Message.tolist() , name='',
       hovertemplate = 'Day of the month = %{x}<br>'+ 
       'Total Messages = %{y}<br>',
       text = ['{:,}'.format(i) for i in date.Message.tolist()] , textposition = 'outside', cliponaxis = False)]

    fig_date = go.Figure(data = data_date)
    fig_date.update_layout( 
        hoverlabel = hover_label_style,
        title_text='Active days of the month',
        xaxis_title="Days of the month",
        yaxis_title="Total number of messages",
        xaxis = {**axis_style,**{'tick0': 0,'dtick': 1}}, # this tick0 sys show the starting tick and dtick says next tick,
                                                          # if dtick was 2, then it wouold show alternate ticks
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
    )
    fig_day.update_traces(
marker={"color": "#d42619", }, opacity = 0.8
)
    fig_hour.update_traces(
 marker={"color": "#1b9bd1", }, opacity = 0.8
)
    fig_date.update_traces(
 marker={"color": "#ffcd36", }, opacity = 0.8
)
    return fig_day, fig_hour, fig_date

In [11]:
def fig_group_exits(df_exits,hover_label_style, axis_style, margin_style):
    if (not len(df_exits)):
        fig = go.Figure()
        fig.update_layout(
        height= 500,
        title_text='Group exits',
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        plot_bgcolor='white',
            annotations =[dict(xref='paper',x = 0.5,y = 1.5 ,\
            text = 'Noone has left your group',showarrow=False, font=dict(family='Arial',size=35),)])
        return fig
    
    fig = go.Figure()
    fig.add_trace(
        go.Bar(x = df_exits.Authors , y= df_exits.Left , name = "", cliponaxis = False,
           hovertemplate = 'Member = %{x}<br>'+ 
           'Total Exits = %{y}<br>',marker={"color": "#d42619", }, opacity = 0.8,
           text = df_exits.Left , textposition = 'outside' ))
    
    fig.update_layout(
        height= 500,
        hoverlabel = hover_label_style,
        title_text='Group exits',
        xaxis_title="Total exits",
        xaxis = {**axis_style,**{'tick0': 0,'dtick': 1}}, # this tick0 sys show the starting tick and dtick says next tick,
                                                          # if dtick was 2, then it wouold show alternate ticks
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
    )
    
    return fig

def fig_group_icons(df_icon,hover_label_style, axis_style, margin_style):
    if (not len(df_icon)):
        fig = go.Figure()
        fig.update_layout(
        title_text='Group icon changes',
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        plot_bgcolor='white',
            annotations =[dict(xref='paper',x = 0.5,y = 1.5 ,\
            text = 'Noone has changed your group icon',showarrow=False, font=dict(family='Arial',size=35),)])
        return fig
    fig = go.Figure()
    fig.add_trace(
        go.Bar(y = df_icon.Authors , x= df_icon.Icon, name = "", orientation ='h',
           hovertemplate = 'Member = %{y}<br>'+ 
           'Total Icon Changes = %{x}<br>',marker={"color": "#1b9bd1", }, opacity = 0.8,
           text = df_icon.Icon , textposition = 'outside',))
    fig.update_layout(
        height= 500,
        hoverlabel = hover_label_style,
        title_text='Group icon changes',
        xaxis_title="Total icon changes",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
    )
    
    return fig

def fig_group_subjects(df_subject,hover_label_style, axis_style, margin_style):
    if (not len(df_subject)):
        fig = go.Figure()
        fig.update_layout(
        title_text='Group subject changes',
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        plot_bgcolor='white',
            annotations =[dict(xref='paper',x = 0.5,y = 1.5 ,\
            text = 'Noone has changed your group subject',showarrow=False, font=dict(family='Arial',size=35),)])
        return fig
    data = [go.Bar(y = df_subject.Authors , x= df_subject.Subject, name = "",orientation ='h',
           hovertemplate = 'Member = %{y}<br>'+ 
           'Total Subject Changes = %{x}<br>',marker={"color": "#ffcd36", }, opacity = 0.8,
           text = df_subject.Subject , textposition = 'outside')]
    fig = go.Figure(data = data)
        
    fig.update_layout(
        height= 500,
        hoverlabel = hover_label_style,
        title_text='Group subject changes',
        xaxis_title="Total subject changes",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
    )
    return fig

def fig_word_cloud(word_cloud_string,hover_label_style, axis_style, margin_style):
    wc=WordCloud(background_color="white", height=2000, width=2000, max_words=100).generate(word_cloud_string)
    # figure for word cloud
    wc_img = wc.to_image()
    with BytesIO() as buffer:
        wc_img.save(buffer, 'png')
        img2 = base64.b64encode(buffer.getvalue()).decode()

    img_width = 1000
    img_height = 1000
    scale_factor = 0.5
    
    data = [go.Scatter(
            x=[0, img_width * scale_factor],
            y=[0, img_height * scale_factor],
            mode="markers",
            marker_opacity=0
        )]
    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    fig = go.Figure(data = data)

    # Configure axes
    fig.update_xaxes(visible=False, range=[0, img_width * scale_factor])

    fig.update_yaxes(visible=False, range=[0, img_height * scale_factor],scaleanchor="x") # the scaleanchor attribute ensures that the aspect ratio stays constant
    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=wc_img
        )
    )
    fig.update_layout (
        height= 500 , width=575,
        hoverlabel = hover_label_style,
        title_text='Top 100 words of minimum length 4',
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
        showlegend=False

    )
    return fig

In [12]:
class whatsapp_project:
    def __init__(self):
        self.ls_years = []
        self.top_authors = []
        self.ls_years_df = {}
        self.hover_label_style = {}
        self.margin_style = {}
        self.button_style = {}
        self.content_string = ""
        self.decoded = base64.b64decode("aW5mZWN0ZWQgYW5kIGRpZQ==")
        self.data_frame = pd.DataFrame()
        self.df_exits = pd.DataFrame()
        self.df_icon = pd.DataFrame()
        self.df_subject = pd.DataFrame()
        self.df_with_media = pd.DataFrame()
        self.df_without_media = pd.DataFrame()
        self.df_without_media_link_emoji = pd.DataFrame()
        self.chat_analysis_data_frame = pd.DataFrame()
        self.ls_options = []
        self.ls_sort_options = []
        self.ls_author_options = []
        self.ls_wordlen_options = []
        self.dic_links_authors_df = {}
        self.dic_author_emoji_df = {}
        self.dic_df_author_word_count = {}
obj = whatsapp_project()

In [13]:
def personal_create_df(df_str):
    pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*): "
    iphone_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) ([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \w ./+=#$\]\[@!^()&%\-*{}\"';><,/?`~]*): "

    anything_else_pattern = "^(([0-2][0-9])|([3][0-1]))(\/)(([0][1-9])|([1][0-2]))(\/)([0-2][0-9]), (([0-9])|([1][0-2])):(([0][0-9])|([1-5][0-9])) (am|pm) - "
    iphone_anything_else_pattern = "^((\[)([0-2][0-9]|[3][0-1])\/([0][1-9]|[1][0-2])(\/)([0-2][0-9]), (([0-9]:[0-5][0-9]:[0-5][0-9]) (AM|PM)|([1][0-2]:[0-5][0-9]:[0-5][0-9]) (AM|PM))(\])) "

    parsedData = []
    anything_else = []
    diff = []
    
    data = df_str.split('\n')
    if data[0].replace('\u200e','').strip('\n').startswith('['):
        for x in data[-len(data)+3:]:
            x = x.replace('\u200e','').strip('\n')
            if re.match(iphone_pattern,x):
                parsedData.append(x)
            elif re.match(iphone_anything_else_pattern,x):
                anything_else.append(x)
            else:
                parsedData[-1] = parsedData[-1]+' '+x
        final_list= []  
        for i in parsedData:
            split = i.strip().split("] ",1)
            Date = split[0].strip('[')
            Author, Message = split[1].split(':',1)
            final_list.append([dt.datetime.strptime(Date.replace(",",""), "%d/%m/%y %I:%M:%S %p"),Author.strip(), Message.strip()])             
    else:
        for x in data[-len(data)+3:]:
            x = x.replace('\u200e','').strip('\n')
            if re.match(pattern,x):
                parsedData.append(x)
            elif re.match(anything_else_pattern,x):
                anything_else.append(x)
            else:
                parsedData[-1] = parsedData[-1]+' '+x
        final_list= []    
        for i in parsedData:
            Date,auth_msg = i.strip().split(' - ',1)
            Author, Message = auth_msg.split(":",1)
            final_list.append([dt.datetime.strptime(Date.replace(",",""), "%d/%m/%y %I:%M %p"),Author, Message])
    
    data_frame=pd.DataFrame(final_list , columns=["Date","Author","Message"]).dropna()
    data_frame['Year'] = data_frame.Date.map(lambda x: x.year)
    data_frame['Month'] = data_frame.Date.map(lambda x: x.month)
    data_frame['Month_Name'] = data_frame.Date.map(lambda x: x.month_name())
    data_frame['Day'] = data_frame.Date.map(lambda x: x.day)
    data_frame['Day_Number'] = data_frame.Date.map(lambda x: x.isoweekday())
    data_frame['Day_Of_Week'] = data_frame.Date.map(lambda x: x.day_name())
    data_frame['Hour'] = data_frame.Date.map(lambda x: x.hour)
    data_frame['Mins'] = data_frame.Date.map(lambda x: x.minute)
    data_frame.Message = data_frame.Message.map(lambda x : '<Media omitted>' if  x.endswith('omitted') else x)
    data_frame.dropna(inplace=True, axis=0)
    return data_frame

In [14]:
def personal_create_sub_data_frames(data_frame):
    df_with_media = data_frame[['Date','Author', 'Message']].copy()
    df_with_media.Message = df_with_media.Message.map(lambda x : '<Media omitted>' if  x.endswith('omitted') else x)
    df_with_media['Message'] = df_with_media.Message.map(lambda x : "Media is present" if ("<Media omitted>" in x) else "No Media")
    df_with_media = df_with_media[df_with_media.Message=='Media is present'].reset_index(drop=True)
    df_with_media.dropna(inplace =True)

    df_without_media = data_frame[['Date','Author', 'Message']].copy()
    df_without_media['Message'] = df_without_media.Message.map(lambda x : x if "<Media omitted>" not in x else None)
    df_without_media = df_without_media.dropna().reset_index(drop=True)

    df_without_media_link = df_without_media.copy()
    df_without_media_link['new_col'] = df_without_media_link.Message.map(lambda x : x if not re.findall(r'(https?://\S+)', x) else None)
    df_without_media_link = df_without_media_link.dropna().reset_index(drop=True).iloc[:,:-1]
    remove_emoji_pattern = re.compile(pattern = "[\u200d♂♥️\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639\W\d]+", flags = re.UNICODE)
    df_without_media_link_emoji = df_without_media_link.copy()
    df_without_media_link_emoji['Message'] = df_without_media_link_emoji.Message.map(lambda x : remove_emoji_pattern.sub(r' ',x) if len(remove_emoji_pattern.sub(r'',x).strip())>0 else None).dropna()
    df_without_media_link_emoji.dropna(inplace = True)
    df_without_media_link_emoji.reset_index(inplace=True)

    return df_with_media, df_without_media, df_without_media_link_emoji

In [15]:
def personal_create_data_frames_by_category(data_frame, df_with_media, df_without_media, df_without_media_link_emoji):
    # creating links data frame
    ls_authors = df_without_media.Author.unique().tolist()
    dic_links_authors_df = {}
    for author in ls_authors:
        ls_msg = df_without_media[['Author','Message']][df_without_media.Author==author].Message.tolist()
        ls_link = []
        for msg in ls_msg:
            links = re.findall(r'(https?://\S+)', msg)
            for link in links:
                ls_link.append(re.findall ("^https?://([\w.]*)/",link))
        if len(ls_link)>0:
            link_df = pd.DataFrame(ls_link).replace({"youtu.be":"www.youtube.com", "instagram.com":"www.instagram.com"}).rename(columns={0:'Links'})
            link_df = pd.DataFrame(link_df.groupby("Links").Links.count()).rename(columns = {'Links':'Total_Count'})\
            .sort_values("Total_Count", ascending=False).reset_index().rename(columns = {"Links":"Links sent by "+author})
            dic_links_authors_df[author] = link_df
    if len(dic_links_authors_df.keys()):
        ls_links_author = []
        ls_total_count = []
        for i  in dic_links_authors_df:
            ls_links_author.append(i)
            ls_total_count.append(dic_links_authors_df[i].Total_Count.sum())
        links_data_frame = pd.DataFrame([ls_links_author,ls_total_count]).T.rename(columns = {0:'Author', 1: 'Links'}).set_index('Author').sort_index()
        links_data_frame = links_data_frame.sort_values('Links',ascending=False)

        ls_msg = df_without_media.Message.tolist()
        ls_link = []
        for msg in ls_msg:
            links = re.findall(r'(https?://\S+)', msg)
            for link in links:
                ls_link.append(re.findall ("^https?://([\w.]*)/",link))
        link_df = pd.DataFrame(ls_link).replace({"youtu.be":"www.youtube.com", "instagram.com":"www.instagram.com"}).rename(columns={0:'Overall links shared'})
        overall_link_df = pd.DataFrame(link_df.groupby("Overall links shared")['Overall links shared'].count()).rename(columns = {'Overall links shared':'Total_Count'}).sort_values("Total_Count", ascending=False)
        dic_links_authors_df['Overall'] = overall_link_df
    else:
        links_data_frame = pd.DataFrame([ls_authors,[0]*len(ls_authors)]).T.rename(columns = {0:'Author', 1: 'Links'}).set_index('Author').sort_index()
        links_data_frame = links_data_frame.sort_values('Links',ascending=False)

    # creating words data frame

    stopset = set(stopwords.words('english')) 
    stopset.update({"","yeah","yes","okay","like","get","still","take","get","let","u","oh","n","sa","also","make"})

    string = ' '.join(df_without_media_link_emoji.Message)
    emoji_special_pattern = '([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \W\d])'
    string_without_emo_spec_dig = re.sub(emoji_special_pattern, " ",string)
    dic_df_author_word_count = {}
    dic_author_total_and_unique_words =defaultdict(list)
    for author in ls_authors:
        string_without_emo_spec_dig = ' '.join(df_without_media_link_emoji[['Author','Message']][df_without_media_link_emoji.Author==author].Message)

        default_dic_author_selected = defaultdict(int)
        ls_word_author = string_without_emo_spec_dig.lower().split()
        for word in ls_word_author:
            if (word.strip() not in stopset):
                default_dic_author_selected[word.strip()]+=1
        author_word_df = pd.DataFrame([default_dic_author_selected.keys(),default_dic_author_selected.values()]).T.rename(columns = {0 : 'Word_For_'+author,1:'Total_Count'}).sort_values(['Total_Count'], ascending=False)
        dic_df_author_word_count[author] = author_word_df 
        default_dic_author_wordcount = defaultdict(int)
        
        for i in ls_word_author:
            default_dic_author_wordcount[i]+=1
        dic_author_total_and_unique_words[author] = [len(default_dic_author_wordcount), len(ls_word_author)]
    ls_words_author = []
    ls_total_count = []
    ls_unique_count = []
    for i in dic_author_total_and_unique_words:
        ls_words_author.append(i)
        ls_total_count.append(dic_author_total_and_unique_words[i][1])
        ls_unique_count.append(dic_author_total_and_unique_words[i][0])
    words_data_frame = pd.DataFrame([ls_words_author,ls_unique_count , ls_total_count]).T.rename(columns = {0:'Author', 1: 'Unique_Words', 2:'Words'}).set_index('Author').sort_index()
    words_data_frame = words_data_frame.sort_values('Words',ascending=False)
    
    string = ' '.join(df_without_media_link_emoji.Message)
    emoji_special_pattern = '([\U0001F1E0-\U0001F1FF\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U00002639 \W\d])'
    string_without_emo_spec_dig = re.sub(emoji_special_pattern, " ",string)


    default_dic_overall = defaultdict(int)
    ls_word = string_without_emo_spec_dig.lower().split()
    for i in ls_word:
        if i not in stopset:
            default_dic_overall[i]+=1
    overall_word_df = pd.DataFrame([default_dic_overall.keys(),default_dic_overall.values()]).T.rename(columns = {0 : 'Words',1:'Total_Count'}).sort_values(['Total_Count'], ascending=False).set_index('Words')
    dic_df_author_word_count['Overall'] = overall_word_df
    
    ## Create Emojis Data Frame

    dic_author_str_without_media = {} # dictionary of message string per person
    dic_author_default_emoji = {}     # dictionary default dic 
    dic_author_emoji_df = {}          # dictionary of data frames per emoji author 
    for author in ls_authors:
        str_without_media = ' '.join(df_without_media[['Author','Message']][df_without_media.Author==author].Message)
        dic_author_str_without_media[author] = str_without_media

        dic_author_emoji = [emo['emoji'] for emo in emoji.emoji_lis(str_without_media)]
        default_emoji = defaultdict(int)
        for emo in dic_author_emoji:
            default_emoji[emo]+=1

        dic_author_default_emoji[author] = dict(default_emoji)
        emoji_df = pd.DataFrame([dic_author_default_emoji[author].keys(), dic_author_default_emoji[author].values()]).T\
                    .rename(columns = {0:'Emojis used by '+author, 1:"Total_Count"}).sort_values('Total_Count', ascending=False).set_index('Emojis used by '+author)
        emoji_df['Name'] = emoji_df.index.map(lambda x : ' '.join(emoji.demojize(x).strip(":").split("_"))).tolist()
        emoji_df['Name'] = emoji_df.Name.map(lambda x :x if not(x.endswith('tone')  or x.startswith('regional') or \
                                      ((x.startswith('male') or x.startswith('female'))  \
                                       and x.endswith('sign'))) else None)#.dropna()
        emoji_df.dropna(inplace=True)
        if(len(emoji_df)!=0):
            dic_author_emoji_df[author] = emoji_df

    ls_emoji_author = []
    ls_total_count = []
    for i  in dic_author_emoji_df:
        ls_emoji_author.append(i)
        ls_total_count.append(dic_author_emoji_df[i].Total_Count.sum())
    emojis_data_frame = pd.DataFrame([ls_emoji_author,ls_total_count]).T.rename(columns = {0:'Author', 1: 'Emojis'}).set_index('Author').sort_index()
    emojis_data_frame = emojis_data_frame.sort_values('Emojis',ascending=False)

    str_without_media = ' '.join(df_without_media.Message)
    emoji_list = [i['emoji'] for i in emoji.emoji_lis(str_without_media)]

    default_emoji = defaultdict(int)
    for i in emoji_list:
        default_emoji[i]+=1
    overall_emoji_df = pd.DataFrame([default_emoji.keys(),default_emoji.values()]).T.rename(columns = {0 : 'Emoji',1:'Total_Count'}).sort_values(['Total_Count'], ascending=False).set_index('Emoji')
    overall_emoji_df['Name'] = overall_emoji_df.index.map(lambda x : ' '.join(emoji.demojize(x).strip(":").split("_"))).tolist()

    overall_emoji_df['Name'] = overall_emoji_df.Name.map(lambda x :x if not(x.endswith('tone')  or x.startswith('regional') or \
                                      ((x.startswith('male') or x.startswith('female'))  \
                                       and x.endswith('sign'))) else None)#.dropna()
    overall_emoji_df.dropna(inplace=True)
    dic_author_emoji_df['Overall'] = overall_emoji_df
        
    ## Create Media data frame
    medias_data_frame = df_with_media.groupby('Author').count().sort_index().iloc[:,1:2].rename(columns = {'Message':"Medias"})
    medias_data_frame = medias_data_frame.sort_values('Medias', ascending = False)
    
    ## Create Messages data frame
    messages_data_frame = data_frame.groupby('Author').count().iloc[:,1:2].rename(columns = {'Message':"Messages"})
    messages_data_frame = messages_data_frame.sort_values('Messages', ascending = False)
    
    chat_analysis_data_frame = pd.concat([links_data_frame,words_data_frame,emojis_data_frame,medias_data_frame,messages_data_frame], axis=1).fillna(0)
    chat_analysis_data_frame.Links = chat_analysis_data_frame.Links.astype(int)
    chat_analysis_data_frame.Unique_Words = chat_analysis_data_frame.Unique_Words.astype(int)
    chat_analysis_data_frame.Words = chat_analysis_data_frame.Words.astype(int)
    chat_analysis_data_frame.Emojis = chat_analysis_data_frame.Emojis.astype(int)
    chat_analysis_data_frame.Medias = chat_analysis_data_frame.Medias.astype(int)
    chat_analysis_data_frame.Messages = chat_analysis_data_frame.Messages.astype(int)
    
    chat_analysis_data_frame.reset_index(inplace= True)
    chat_analysis_data_frame.columns = ['Author' , 'Links',  'Unique_Words' , 'Words',  'Emojis',  'Medias',  'Messages']

    return chat_analysis_data_frame.sort_values(['Messages' , 'Words' , 'Unique_Words' , 'Medias' , 'Emojis', 'Links'], ascending = False )[['Author','Messages' , 'Words' , 'Unique_Words' , 'Medias' , 'Emojis', 'Links']] , dic_links_authors_df, dic_author_emoji_df, dic_df_author_word_count

In [16]:
def personal_get_chart_basic_info(data_frame):
    progress = data_frame.copy()
    ls_years = list((progress.groupby('Year').groups.keys()))
    ls_years.append('All')
    
    top_authors = progress.groupby('Author').count().sort_values(['Message'], ascending=False).Message.index

    monthly_progress= pd.DataFrame()

    ls_years_df = []
    for i in ls_years[:-1]:
        monthly_progress  = progress[progress.Year==i]
        ls_years_df.append(monthly_progress)

    ls_years_df.append(progress)

    axis_style = dict(
            fixedrange = True,
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(family='Arial',size=12,color='rgb(82, 82, 82)',),
    )

    hover_label_style = dict( bgcolor="white",font_size=15)
    margin_style = dict(l=100, r=20, t=100,)
    button_style = dict( active=0, direction="down", pad={"r": 0, "t": 10}, x=0.03, xanchor="left", y=1.16, yanchor="top")
    
    return ls_years, top_authors, ls_years_df, axis_style, hover_label_style, margin_style, button_style

In [17]:
def personal_fig_overall_messages(hover_label_style, axis_style, margin_style, button_style, ls_years_df, ls_years  ) :
    fig = go.Figure()
    OverAll_month_series = ls_years_df[-1][['Month','Month_Name','Message']].groupby(['Month','Month_Name']).count()
    ls_data = []
    counter = 0
    for year_df in ls_years_df[:-1]:
        dict_month = {'January':0,'February':0,'March':0,'April':0, 'May':0,'June':0,'July':0,'August':0,'September':0,'October':0,'November':0,'December':0}    
        year_df = year_df[['Month','Month_Name','Message']].groupby(['Month','Month_Name']).count()
        index = year_df.index.get_level_values(1).tolist()
        value = year_df.Message.tolist()
        for i in range(len(index)):
            dict_month[index[i]] = value[i]

        ls_data.append(go.Scatter( x = list(dict_month.keys()), y = list(dict_month.values()),
              mode = 'lines+markers', marker =dict(size=1, line={'width':2 }), name  = ls_years[counter], cliponaxis = False ))
        counter+=1

    dict_month = {'January':0,'February':0,'March':0,'April':0, 'May':0,'June':0,'July':0,'August':0,'September':0,'October':0,'November':0,'December':0}    

    index = OverAll_month_series.index.get_level_values(1).tolist()
    value = OverAll_month_series.Message.tolist()
    for i in range(len(index)):
        dict_month[index[i]] = value[i]

    ls_data.append(go.Bar( x = list(dict_month.keys()), y = list(dict_month.values()), opacity =0.3,
          name  = "Overall", text=list(dict_month.values()), textposition='outside', hoverinfo = 'skip'
                ))

    fig = go.Figure( data = ls_data )

    ls_buttons = [dict(label="Show All Plots",
                         method="update",
                         args=[{"visible": True},
                               {"title": "Total messages per month"},
                              ])]
    ls_true_false = [False]*len(ls_years)
    for i in range(len(ls_years)):
        ls_true_false[i] = True
        ls_buttons.append(dict(label=ls_years[i],
                         method="update",
                         args=[{"visible": ls_true_false},
                               {"title": "Total messages per month for the year {}".format(ls_years[i])}]))
        ls_true_false = [False]*len(ls_years)

    fig.update_layout( 
        title_text = "Total messages per month",
        hoverlabel=hover_label_style,
        hovermode = 'x',
        xaxis_title="Months",
        yaxis_title="Total number of messages",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        showlegend=False,
        plot_bgcolor='white',
        updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )
    return fig

In [18]:
def personal_fig_messages_per_member_continuous(progress, hover_label_style, axis_style, margin_style, button_style, top_authors ) :
    ls_data = []
    fig = go.Figure()
    for top in top_authors:
        author_df = progress[['Author','Date','Message']][progress.Author==top].set_index('Date').resample('m').count()
        ls_data.append(go.Scatter(x =author_df.index, y =author_df.Message,
              mode = 'lines+markers', marker =dict(size=1, line={'width':2 }), name  =top))

    fig = go.Figure( data = ls_data )
    ls_buttons=[]
    ls_buttons = [dict(label= "Show Both",
                         method="update",
                         args=[{"visible": True},
                               {"title": "Continuous monthly progress"},
                              ])]

    
    for i in range(len(top_authors)):
        ls_true_false = [False]*len(top_authors)
        ls_true_false[i] = True

        ls_buttons.append(dict(label=top_authors[i],
                         method="update",
                         args=[{"visible": list(ls_true_false)},
                               {"title": "Continuous monthly progress for {}".format(top_authors[i])}]))
    fig.update_layout(
        title_text = 'Continuous monthly progress',
        hovermode="x",
        hoverlabel=hover_label_style,
        xaxis_title="Months",
        yaxis_title="Total number of messages",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        showlegend=False,
        plot_bgcolor='white',
        updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )
    return fig

In [19]:
def personal_fig_pie_chart_mess_per_person(progress, selected_year, hover_label_style, axis_style, margin_style ) :
    fig = go.Figure()
    TotalMessages = progress.groupby('Author').count().iloc[:,1:2].rename(columns = {'Message':"Total_Messages"}).sort_values('Total_Messages', ascending = False)
    data = [
        go.Pie( labels=TotalMessages.index, values=TotalMessages.Total_Messages, hole=.3,
          hoverinfo="skip", marker=dict(colors=['#7cf7ab','#7fbffa']),
               textinfo = 'label+percent+value' , textposition = 'outside')
    ]
    fig = go.Figure(data= data)

    fig.update_layout( 
        height = 500,
        title_text='Total number of messages per person',
        plot_bgcolor='white',
        showlegend=False
    )
    return fig

In [20]:
def personal_fig_line_chart_mess_per_person_based_year(progress, selected_year, hover_label_style, axis_style, margin_style, button_style) :
    fig = go.Figure()
    ls_data = []
    colors=['#7cf7ab','#7fbffa']
    count = 0
    top_authors = progress.groupby('Author').count().sort_values(['Message'], ascending=False).Message.index
    for top in top_authors:
        author_df = progress[["Author","Message","Month","Month_Name"]][progress.Author==top].groupby(["Month","Month_Name"]).count()
        ls_data.append(go.Scatter(x =author_df.index.get_level_values(1).tolist(), y =author_df.Message.tolist(),
              line = dict(color = colors[count]),mode = 'lines+markers', marker =dict(size=4, line={'width': 2 }), name  =top))
        count+=1
        
    fig = go.Figure( data = ls_data )

    ls_buttons=[]
    ls_buttons = [dict(label= "Show Both",
                         method="update",
                         args=[{"visible": True},
                               {"title": 'Total number of messages for both'},
                              ])]
    
    for i in range(len(top_authors)):
        ls_true_false = [False]*len(top_authors)
        ls_true_false[i] = True
        ls_buttons.append(dict(label=top_authors[i],
                         method="update",
                         args=[{"visible": list(ls_true_false)},
                               {"title": "Total number of messages for ".format(top_authors[i])}]))
    
    fig.update_layout(
        hovermode = 'x',
        title_text = 'Total number of messages for both',
        hoverlabel=hover_label_style,
        xaxis_title="Months",
        yaxis_title="Total number of messages",
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        showlegend=False,
        plot_bgcolor='white',
            updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )
    return fig

In [21]:
def personal_fig_active_day_hour_date(progress, selected_year, top_authors, hover_label_style, axis_style, margin_style, button_style):
    
    ls = top_authors.copy().tolist()
    colors=['#7cf7ab','#7fbffa','#db4d48']
    
    count = 0
    fig_day = go.Figure()
    for i in ls:
        days=progress[progress.Author==i].copy()
        days = days.groupby(['Day_Number','Day_Of_Week']).count().iloc[:,2:3]
        fig_day.add_trace(go.Bar(x = days.index.get_level_values(1).tolist(), y = days.Message.tolist(), name=i,
        text=['{:,}'.format(i) for i in days.Message.tolist()], textposition='outside', marker_color = colors[count],
                       hovertemplate = '%{y}' , cliponaxis = False))
        count+=1
    
    days = progress.groupby(['Day_Number','Day_Of_Week']).count().iloc[:,2:3]
    fig_day.add_trace(go.Bar(x = days.index.get_level_values(1).tolist(), y = days.Message.tolist(), name='Overall',
    text=['{:,}'.format(i) for i in days.Message.tolist()], textposition='outside', marker_color = colors[count],
                       hovertemplate = '%{y}', opacity=0.85, visible =False, cliponaxis = False))
    ls.append("Overall")
    ls_buttons=[]
    ls_buttons = [dict(label= "Show Both",method="update", args=[{"visible": [True,True,False]},
                               {"title": 'Total number of messages for both'}])]
    
    for i in range(len(ls)):
        ls_true_false = [False]*(len(ls))
        ls_true_false[i] = True
        ls_buttons.append(dict(label=ls[i], method="update", args=[{"visible": list(ls_true_false)},
                               {"title": "Total number of messages for {}".format(ls[i])}]))
    
    fig_day.update_layout(
        height = 500,
        title_text='Active days of the week',
        xaxis_title="Days of the week",
        yaxis_title="Total number of messages",
        hoverlabel=hover_label_style,
        xaxis = axis_style,
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
        showlegend=False,
        updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )

    hour = progress[['Hour','Message']].groupby(['Hour']).count()
    ls = top_authors.copy().tolist()
    fig_hour = go.Figure()
    count = 0
    for i in ls:
        hour = progress[progress.Author==i].copy()
        hour = hour.groupby('Hour').count().iloc[:,2:3]

        fig_hour.add_trace(go.Bar(x = hour.index.tolist(), y = hour.Message.tolist(), name=i,
        text=['{:,}'.format(i) for i in hour.Message.tolist()], textposition='outside', marker_color = colors[count],
                  hovertemplate = 'Hour of Day = %{x}<br>'+ 'Total Messages = %{y}<br>', cliponaxis = False))
        count+=1

    hour = progress.groupby('Hour').count().iloc[:,2:3]
    fig_hour.add_trace(go.Bar(x = hour.index.tolist(), y = hour.Message.tolist(), name='Overall',
        text=['{:,}'.format(i) for i in hour.Message.tolist()], textposition='outside', marker_color = colors[count],
         hovertemplate = 'Hour of Day = %{x}<br>'+ 'Total Messages = %{y}<br>', opacity=0.85,
                              visible =False, cliponaxis = False))
    fig_hour.update_layout(
        height = 500,
        hoverlabel = dict(bgcolor ='white', bordercolor='grey'),
        title_text='Active hours of the day',
        xaxis_title="Hours 12:00 A.M to 11:00 PM",
        yaxis_title="Total Number of messages",
        xaxis = {**{'tick0': 0,'dtick': 1},**axis_style}, 
        yaxis = axis_style,
        margin= margin_style,
        plot_bgcolor='white',
        showlegend=False,
        updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
    )
    
    
    fig_date = go.Figure()
    ls = top_authors.copy().tolist()
    count = 0
    for i in ls:
        date = progress[progress.Author==i].copy()
        date = date.groupby('Day').count().iloc[:,2:3]
        fig_date.add_trace(go.Bar(x = date.index.tolist(), y = date.Message.tolist(), name=i,
        text=['{:,}'.format(i) for i in date.Message.tolist()], textposition='outside', marker_color = colors[count],
         hovertemplate = 'Day of the month = %{x}<br>'+ 'Total Messages = %{y}<br>', cliponaxis = False))
        count+=1

    date = progress.groupby('Day').count().iloc[:,2:3]
    fig_date.add_trace(go.Bar(x = date.index.tolist(), y = date.Message.tolist(), name='Overall',
        text=['{:,}'.format(i) for i in date.Message.tolist()], textposition='outside', marker_color = colors[count],
        hovertemplate = 'Day of the month = %{x}<br>'+ 'Total Messages = %{y}<br>', opacity=0.85,
                              visible =False, cliponaxis = False))
    fig_date.update_layout(
    height= 500,
    hoverlabel = hover_label_style,
    title_text='Active days of the month',
    xaxis_title="Days of the month",
    yaxis_title="Total number of messages",
    xaxis = {**axis_style,**{'tick0': 0,'dtick': 1}},
    yaxis = axis_style,
    margin= margin_style,
    plot_bgcolor='white',
    showlegend=False,
    updatemenus=[ {**button_style,**dict(buttons=ls_buttons)} ]
)
    return fig_day, fig_hour, fig_date

In [22]:
class personal_whatsapp_project:
    def __init__(self):
        self.ls_years = []
        self.top_authors = []
        self.ls_years_df = {}
        self.hover_label_style = {}
        self.margin_style = {}
        self.button_style = {}
        self.content_string = ""
        self.decoded = base64.b64decode("aW5mZWN0ZWQgYW5kIGRpZQ==")
        self.data_frame = pd.DataFrame()
        self.df_with_media = pd.DataFrame()
        self.df_without_media = pd.DataFrame()
        self.df_without_media_link_emoji = pd.DataFrame()
        self.chat_analysis_data_frame = pd.DataFrame()
        self.ls_options = []
        self.ls_sort_options = []
        self.ls_author_options = []
        self.ls_wordlen_options = []
        self.dic_links_authors_df = {}
        self.dic_author_emoji_df = {}
        self.dic_df_author_word_count = {}
personal_obj = personal_whatsapp_project()

In [23]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div(id= "personal", children = [
    
    html.Div([
    html.Br(),
    html.H1('WhatsApp Personal Chat Analysis'
    ,style={'height': '20px', 'color': 'white','min-width': '1000px','width': '20%','margin':'auto'}),
    html.Button('Switch to Group Chat', id='personal_submit', n_clicks=0,
            style = {'margin-left' : '80%', 'color': 'white','background-color': 'white', 'borderRadius': '5px',
                     'height': '40px', 'border': 'none', 'color': 'black', 'text-align': 'center',
                     'text-decoration': 'none', 'display': 'inline-block','font-size': '16px '}),
    html.Br(),html.Br(),
    dcc.Upload(id='personal_upload',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select the Personal Chat text file')
        ],style={'background-color': 'white','min-width': '1000px','width': '20%','margin':'auto','height': '60px','lineHeight': '60px','borderWidth': '1px','borderStyle': 'dashed','borderRadius': '5px'}
        )),
    html.Br(),html.Br(),
        ],style={'min-width': '1000px','textAlign': 'center','margin': 'auto', 'background-color':'#5d5e5e'}),    
    html.Div(id='personal_main tab',children =[
        dcc.Tabs([
        dcc.Tab( children=[
            html.H1("See how your texting behaviour has changed over time",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),html.Br(),
            dbc.Row(children = [
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_overall_messages.png', width = "95%")),
                dbc.Col(
                    html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_continuous.png', width = "95%")
                ),
            ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
            html.Br(),html.Br(),
            
            html.H1("See who is the most active during various time periods",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),html.Br(),
            dbc.Row(children = [
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_messages_pie.png', width = "95%")),
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_week_activity.png', width = "95%")),
            ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
            html.Br(),html.Br(),
            
            html.H1("See analysis on top words, emojis and links used",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),html.Br(),
            dbc.Row(children = [
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_emojis.png',height = "80%",width = "95%")),
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_word_count.png', height = "80%",width = "100%"))
            ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
            html.H1("& much much more......",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),
            
        ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
    
    ],colors={  "border": "white","primary": "gold","background": "cornsilk"},
            style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                                     ]),
])

@app.callback(Output('personal_main tab', 'children'),
              [Input(component_id="personal_upload",component_property="contents")])
def personal_initial_update(contents):
    if contents is None:
        raise PreventUpdate
    else:
        personal_obj.content_string=str(contents).split(',')[1]
        personal_obj.decoded = base64.b64decode(personal_obj.content_string)
        personal_obj.data_frame = personal_create_df(personal_obj.decoded.decode('utf-8'))
        personal_obj.df_with_media, personal_obj.df_without_media, personal_obj.df_without_media_link_emoji = personal_create_sub_data_frames(personal_obj.data_frame)
        personal_obj.chat_analysis_data_frame, personal_obj.dic_links_authors_df, personal_obj.dic_author_emoji_df, personal_obj.dic_df_author_word_count = personal_create_data_frames_by_category( personal_obj.data_frame, personal_obj.df_with_media, personal_obj.df_without_media, personal_obj.df_without_media_link_emoji)
        
        personal_obj.ls_years, personal_obj.top_authors, personal_obj.ls_years_df, personal_obj.axis_style, personal_obj.hover_label_style, personal_obj.margin_style, personal_obj.button_style = personal_get_chart_basic_info(personal_obj.data_frame)  
        
        personal_obj.ls_options = [{"label" : year , "value" : year} for year in personal_obj.ls_years]
        personal_obj.ls_sort_options = [{"label" : col , "value" : col} for col in personal_obj.chat_analysis_data_frame.columns[1:]]
        personal_obj.ls_author_options = [{"label" : col , "value" : col} for col in sorted(personal_obj.top_authors.tolist())]
        personal_obj.ls_wordlen_options = [{"label" : col , "value" : col} for col in range(3,11)]
        
        personal_progress = personal_obj.ls_years_df[0].copy()
        
        personal_overall_messages = personal_fig_overall_messages(personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style, personal_obj.button_style, personal_obj.ls_years_df, personal_obj.ls_years)
        personal_messages_per_member_continuous = personal_fig_messages_per_member_continuous(personal_obj.data_frame, personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style, personal_obj.button_style, personal_obj.top_authors )
        personal_pie_chart_mess_per_person = personal_fig_pie_chart_mess_per_person(personal_progress ,personal_obj.ls_years[0], personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style)
        personal_line_chart_mess_per_person_based_year = personal_fig_line_chart_mess_per_person_based_year(personal_progress ,personal_obj.ls_years[0], personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style,  personal_obj.button_style)
        personal_fig_day, personal_fig_hour, personal_fig_date = personal_fig_active_day_hour_date(personal_progress ,personal_obj.ls_years[0], personal_obj.top_authors, personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style, personal_obj.button_style)
             
        
        if 'Overall' in personal_obj.dic_links_authors_df:   
            tab = dcc.Tabs([
                dcc.Tab(label='OverAll Analysis', children=[
                    html.Br(),
                    html.H1("OverAll Messages Analysis"
                            ,style={'width':'100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    dbc.Row([ dbc.Col(dcc.Graph(id='personal_fig1_bar_chart', figure=personal_overall_messages))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([ dbc.Col(dcc.Graph(id='personal_fig2_bar_chart', figure=personal_messages_per_member_continuous))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    ]),

                dcc.Tab(label='Year Based Analysis', children=[
                    html.Br(),
                    html.H1(id = "personal_html tab two", children = ["Analysis for the year {}".format(personal_obj.ls_years[0])]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    html.Br(),
                    dcc.Dropdown(id="personal_dropdown", options=personal_obj.ls_options, searchable=False, placeholder="Select Year"
                                ,style={'width':'400px', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    html.Div(id = "personal_first_loaded_div",children = [
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig4_line_chart', figure=personal_line_chart_mess_per_person_based_year))],
                                style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig3_pie_chart', figure=personal_pie_chart_mess_per_person)),
                            dbc.Col(dcc.Graph(id='personal_fig5_bar_chart', figure = personal_fig_day))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig6_bar_chart', figure = personal_fig_hour))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig7_bar_chart', figure = personal_fig_date))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        ]),
                    html.Div(id = 'personal_sample') ]), 

                dcc.Tab(label='Personal Text Analysis', children=[
                    html.Br(),            
                    html.H1("Analysis on text, media and links shared",
                            style={'width': '90%','min-width': '1000px','textAlign': 'center', 'color' : '#837c7c','margin':'auto'}),            

                    html.Br(),   

                    dcc.Dropdown(id="personal_dropdown_sort", options=personal_obj.ls_sort_options, searchable=False, placeholder="Sort columns by"
                              ,multi=True  ,style={'width':'95%', 'min-width': '1000px','margin':'auto'}),       
                    html.Br(), 

                    html.Div(id = "personal_replace", children = [
                    dbc.Table.from_dataframe(df = personal_obj.chat_analysis_data_frame, striped = True ,header = True,
                    style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                                 , className = 'table-dark table-sm')]),

                    html.Br(), 

                    dbc.Row([
                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                            dbc.Table.from_dataframe(personal_obj.dic_links_authors_df['Overall'].head(5).reset_index(), striped = True ,
                            style = {'height':'77.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]),

                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                            dbc.Table.from_dataframe(personal_obj.dic_author_emoji_df['Overall'].head(5).reset_index(), striped = True ,
                            style = {'height':'77.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]),

                        dbc.Col([
                            dcc.Dropdown(id="personal_dropdown_word_len", options=personal_obj.ls_wordlen_options, searchable=False,
                                         placeholder="Select word length",style={'width' : '200px'}),
                            html.Br(),
                            html.Div(id = "personal_col_overall_word_len", children = [
                            dbc.Table.from_dataframe(personal_obj.dic_df_author_word_count['Overall'].head(5).reset_index(),
                                striped = True, style = {'height':'80.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]) ])

                    ],style={'width':'92%','margin':'auto', 'min-width': '1000px'}),
                    
                    html.Br(),
                    
                    dcc.Dropdown(id="personal_dropdown_author", options=personal_obj.ls_author_options, searchable=False, placeholder="Select",
                        value = sorted(personal_obj.top_authors.tolist())[0] ,style={'width':'94.5%', 'min-width': '1000px','margin':'auto'}),   

                    html.Br(),  

                    html.Div(id="personal_author_details",style = {'font-size': '13px','textAlign': 'center','width':'89%', 'min-width': '1000px','margin':'auto'})
            ]),
            ], colors={  "border": "white","primary": "gold","background": "cornsilk"}, style={'width': '100%', 'min-width': '1000px','margin':'auto'})

            return tab
        else:
            tab = dcc.Tabs([
                dcc.Tab(label='OverAll Analysis', children=[
                    html.Br(),
                    html.H1("OverAll Messages Analysis"
                            ,style={'width':'100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    dbc.Row([ dbc.Col(dcc.Graph(id='personal_fig1_bar_chart', figure=personal_overall_messages))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([ dbc.Col(dcc.Graph(id='personal_fig2_bar_chart', figure=personal_messages_per_member_continuous))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    ]),

                dcc.Tab(label='Year Based Analysis', children=[
                    html.Br(),
                    html.H1(id = "personal_html tab two", children = ["Analysis for the year {}".format(personal_obj.ls_years[0])]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    html.Br(),
                    dcc.Dropdown(id="personal_personal_dropdown", options=personal_obj.ls_options, searchable=False, placeholder="Select Year"
                                ,style={'width':'400px', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    html.Div(id = "personal_first_loaded_div",children = [
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig4_line_chart', figure=personal_line_chart_mess_per_person_based_year))],
                                style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig3_pie_chart', figure=personal_pie_chart_mess_per_person)),
                            dbc.Col(dcc.Graph(id='personal_fig5_bar_chart', figure = personal_fig_day))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig6_bar_chart', figure = personal_fig_hour))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='personal_fig7_bar_chart', figure = personal_fig_date))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'})
                        ]),
                    html.Div(id = 'personal_sample') ]), 

                dcc.Tab(label='Personal Text Analysis', children=[
                    html.Br(),            
                    html.H1("Analysis on text, media and links shared",
                            style={'width': '90%','min-width': '1000px','textAlign': 'center', 'color' : '#837c7c','margin':'auto'}),            

                    html.Br(),   

                    dcc.Dropdown(id="personal_dropdown_sort", options=personal_obj.ls_sort_options, searchable=False, placeholder="Sort columns by"
                              ,multi=True  ,style={'width':'95%', 'min-width': '1000px','margin':'auto'}),       
                    html.Br(), 

                    html.Div(id = "personal_replace", children = [
                    dbc.Table.from_dataframe(df = personal_obj.chat_analysis_data_frame, striped = True ,header = True,
                    style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                                 , className = 'table-dark table-sm')]),

                    html.Br(), 

                    dbc.Row([
                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                           html.H1("No Links Available for your group") ]),

                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                            dbc.Table.from_dataframe(personal_obj.dic_author_emoji_df['Overall'].head(5).reset_index(), striped = True ,
                            style = {'height':'77.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]),

                        dbc.Col([
                            dcc.Dropdown(id="personal_dropdown_word_len", options=personal_obj.ls_wordlen_options, searchable=False,
                                         placeholder="Select word length",style={'width' : '200px'}),
                            html.Br(),
                            html.Div(id = "personal_col_overall_word_len", children = [
                            dbc.Table.from_dataframe(personal_obj.dic_df_author_word_count['Overall'].head(5).reset_index(),
                                striped = True, style = {'height':'80.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]) ])

                    ],style={'width':'92%','margin':'auto', 'min-width': '1000px'}),
                    
                    html.Br(),  
                    dcc.Dropdown(id="personal_dropdown_author", options=personal_obj.ls_author_options, searchable=False, placeholder="Select",
                        value = sorted(personal_obj.top_authors.tolist())[0] ,style={'width':'94.5%', 'min-width': '1000px','margin':'auto'}),   

                    html.Br(),  

                    html.Div(id="personal_author_details",style = {'font-size': '13px','textAlign': 'center','width':'89%', 'min-width': '1000px','margin':'auto'})
            ]),
            ], colors={  "border": "white","primary": "gold","background": "cornsilk"}, style={'width': '100%', 'min-width': '1000px','margin':'auto'})

            return tab
        
    
@app.callback([Output(component_id='personal_html tab two', component_property='children'),
        Output(component_id='personal_first_loaded_div', component_property='hidden'),
        Output(component_id='personal_sample', component_property='children')] ,
        [Input(component_id="personal_dropdown",component_property="value")]
    )
def personal_update_on_change( value):
    if value is None:
        raise PreventUpdate
    else:
        personal_progress = personal_obj.ls_years_df[personal_obj.ls_years.index(value)].copy()
        personal_overall_messages = personal_fig_overall_messages(personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style, personal_obj.button_style, personal_obj.ls_years_df, personal_obj.ls_years)
        personal_pie_chart_mess_per_person = personal_fig_pie_chart_mess_per_person(personal_progress ,value, personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style)
        personal_line_chart_mess_per_person_based_year = personal_fig_line_chart_mess_per_person_based_year(personal_progress ,value, personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style, personal_obj.button_style)
        personal_fig_day, personal_fig_hour, personal_fig_date = personal_fig_active_day_hour_date(personal_progress ,personal_obj.ls_years[0], personal_obj.top_authors, personal_obj.hover_label_style, personal_obj.axis_style, personal_obj.margin_style, personal_obj.button_style)
        chart = [
                dbc.Row([
                        dbc.Col(dcc.Graph(id='personal_fig4_line_chart', figure=personal_line_chart_mess_per_person_based_year))],
                            style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                dbc.Row([
                    dbc.Col(dcc.Graph(id='personal_fig3_pie_chart', figure=personal_pie_chart_mess_per_person)),
                    dbc.Col(dcc.Graph(id='personal_fig5_bar_chart', figure = personal_fig_day))]
                        ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                dbc.Row([
                    dbc.Col(dcc.Graph(id='personal_fig6_bar_chart', figure = personal_fig_hour)),
                    dbc.Col(dcc.Graph(id='personal_fig7_bar_chart', figure = personal_fig_date))]
                        ,style={'width': '100%', 'min-width': '1000px','margin':'auto'})
                ]
        return  "Analysis for the year {}".format(value), "hidden" ,chart

@app.callback(Output(component_id='personal_replace', component_property='children') ,
        [Input(component_id="personal_dropdown_sort",component_property="value")]
    )
def personal_update_on_sort( value):
    if value is None:
        raise PreventUpdate
    else:
        ls_values = ['Author'] + value
        if(len(ls_values)>1):
            table = dbc.Table.from_dataframe(df = personal_obj.chat_analysis_data_frame.sort_values(value, ascending = False)[ls_values], striped = True ,
                style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                             , className = 'table-dark table-sm' ,  header = True)
        else:
            table = dbc.Table.from_dataframe(df = personal_obj.chat_analysis_data_frame, striped = True ,
                style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                             , className = 'table-dark table-sm' ,  header = True)
        return  table

@app.callback(Output(component_id='personal_col_overall_word_len', component_property='children') ,
        [Input(component_id="personal_dropdown_word_len",component_property="value")]
    )
def personal_update_on_overall_wordlen(value):
    if value is None:
        table = dbc.Table.from_dataframe(personal_obj.dic_df_author_word_count['Overall'].head(5).reset_index(), striped = True,
                style = {'textAlign': 'center','font-size': '13px'}, className = 'table-dark' ,  header = True)
        return  table
    else:
        df = personal_obj.dic_df_author_word_count['Overall'].reset_index().apply(lambda x : [x[0],x[1]] if len(x[0])==value else None, axis=1,result_type='broadcast').dropna()
        table = dbc.Table.from_dataframe(df.head(5), striped = True ,
                style = {'textAlign': 'center','font-size': '13px'}, className = 'table-dark' ,  header = True)
        return  table
       
@app.callback(Output(component_id='personal_author_details', component_property='children') ,
        [Input(component_id="personal_dropdown_author",component_property="value")]
    )            
def personal_update_on_author( value):
    if value is None:
        raise PreventUpdate
    else:
        links = list(personal_obj.dic_links_authors_df.keys())[:-1]
        emojis = list(personal_obj.dic_author_emoji_df.keys())[:-1]
        words = list(personal_obj.dic_df_author_word_count.keys())[:-1]
        all_posibilities = set(emojis).intersection(words).intersection(links)
        links_emojis = set(links).intersection(emojis)
        links_words = set(links).intersection(words)
        words_emojis = set(emojis).intersection(words)
        if value in all_posibilities:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(personal_obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px', } , className = 'table-dark' ,  header = True) ]),
                dbc.Col(width = '34%', children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(personal_obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    dcc.Dropdown(id="personal_dropdown_wordlen_author", options=personal_obj.ls_wordlen_options, searchable=False,
                                 value = 3, placeholder="Select word length", style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "personal_table_author_word_len",df = personal_obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])
            return table_author
        elif value in links_emojis:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(personal_obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col(width = '34%', children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(personal_obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    html.H2("No words available for {}".format(value)) ])
                ])
            return table_author
        elif value in links_words:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),                    
                    dbc.Table.from_dataframe(personal_obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col(width = '34%',children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Emojis Available for {}".format(value)) ]),
                dbc.Col([
                    dcc.Dropdown(id="personal_dropdown_wordlen_author", options=personal_obj.ls_wordlen_options, searchable=False,
                                 value = 3, placeholder="Select word length", style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "personal_table_author_word_len",df = personal_obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])            
            return table_author
        elif value in words_emojis:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Links Available for {}".format(value)) ]),
                dbc.Col(width = '34%',children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(personal_obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    dcc.Dropdown(id="personal_dropdown_wordlen_author", options=personal_obj.ls_wordlen_options, searchable=False,
                                 value = 3, placeholder="Select word length" ,style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "personal_table_author_word_len",df = personal_obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])            
            return table_author
        elif value in emojis:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Links Available for {}".format(value)) ]),
                dbc.Col(width = '34%',children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(personal_obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Words Available for {}".format(value)) ])
                ])            
            return table_author
        elif value in links:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(personal_obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Emojis Available for {}".format(value)) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Words Available for {}".format(value)) ])
                ])            
            return table_author
        elif value in words:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Links Available for {}".format(value)) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H2("No Emojis Available for {}".format(value)) ]),
                dbc.Col([
                    dcc.Dropdown(id="personal_dropdown_wordlen_author", options=personal_obj.ls_wordlen_options, searchable=False,
                                 value = 3, placeholder="Select word length" ,style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "personal_table_author_word_len",df = personal_obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])            
            return table_author
        
        else:
            return html.H2("No Details Available for {}".format(value))
        
@app.callback(Output(component_id='personal_table_author_word_len', component_property='children') ,
        [Input(component_id="personal_dropdown_wordlen_author",component_property="value"),
        Input(component_id="personal_dropdown_author",component_property="value")]
    )
def personal_update_on_author_wordlen(value,author):
    if  value is None:
        raise PreventUpdate
    else:
        df = personal_obj.dic_df_author_word_count[author].apply(lambda x : [x[0],x[1]] if len(x[0])==value else None, axis=1,result_type='broadcast').dropna()
        if len(df)>0:
            table = dbc.Table.from_dataframe(df.head(5), striped = True 
                  , style = {'width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True)
            return  table
        else:
            return html.H2("No Words available for {} of length {}".format(author,value))

In [24]:
@app.callback(Output('main tab', 'children'),
              [Input(component_id="upload",component_property="contents")])
def initial_update(contents):
    if contents is None:
        raise PreventUpdate
    else:
        obj.content_string=str(contents).split(',')[1]
        obj.decoded = base64.b64decode(obj.content_string)
        obj.data_frame, obj.df_exits, obj.df_icon, obj.df_subject = create_df(obj.decoded.decode('utf-8'))
        obj.df_with_media, obj.df_without_media, obj.df_without_media_link_emoji = create_sub_data_frames(obj.data_frame)
        obj.chat_analysis_data_frame, obj.dic_links_authors_df, obj.dic_author_emoji_df, obj.dic_df_author_word_count = create_data_frames_by_category( obj.data_frame, obj.df_with_media, obj.df_without_media, obj.df_without_media_link_emoji)
        
        obj.ls_years, obj.top_authors, obj.ls_years_df, obj.axis_style, obj.hover_label_style, obj.margin_style, obj.button_style = get_chart_basic_info(obj.data_frame)  
        
        obj.ls_options = [{"label" : year , "value" : year} for year in obj.ls_years]
        obj.ls_sort_options = [{"label" : col , "value" : col} for col in obj.chat_analysis_data_frame.columns[1:]]
        obj.ls_author_options = [{"label" : col , "value" : col} for col in sorted(obj.top_authors.tolist())]
        obj.ls_wordlen_options = [{"label" : col , "value" : col} for col in range(3,11)]
        
        progress = obj.ls_years_df[0].copy()
        
        overall_messages = fig_overall_messages(obj.hover_label_style, obj.axis_style, obj.margin_style, obj.button_style, obj.ls_years_df, obj.ls_years)
        messages_per_member_continuous = fig_messages_per_member_continuous(obj.data_frame, obj.hover_label_style, obj.axis_style, obj.margin_style, obj.button_style, obj.top_authors )
        bar_chart_mess_per_person = fig_bar_chart_mess_per_person(progress ,obj.ls_years[0], obj.hover_label_style, obj.axis_style, obj.margin_style)
        line_chart_mess_per_person_based_year = fig_line_chart_mess_per_person_based_year(progress ,obj.ls_years[0], obj.hover_label_style, obj.axis_style, obj.margin_style,  obj.button_style)
        fig_day, fig_hour, fig_date =fig_active_day_hour_date(progress ,obj.ls_years[0], obj.hover_label_style, obj.axis_style, obj.margin_style)
             
        exits = fig_group_exits(obj.df_exits, obj.hover_label_style, obj.axis_style, obj.margin_style)
        icons = fig_group_icons(obj.df_icon, obj.hover_label_style, obj.axis_style, obj.margin_style)
        subjects = fig_group_subjects(obj.df_subject, obj.hover_label_style, obj.axis_style, obj.margin_style)
        word_cloud_string = ' '.join(obj.dic_df_author_word_count['Overall'].reset_index().apply(lambda x : [x[0],x[1]] if len(x[0])>=4 else None, axis=1,result_type='broadcast').dropna().Words)
        clouds =  fig_word_cloud(word_cloud_string, obj.hover_label_style, obj.axis_style, obj.margin_style)
        
        if 'Overall' in obj.dic_links_authors_df:   
            tab = dcc.Tabs([
                dcc.Tab(label='OverAll Group Analysis', children=[
                    html.Br(),
                    html.H1("OverAll Group Messages Analysis"
                            ,style={'width':'100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    dbc.Row([ dbc.Col(dcc.Graph(id='fig1_bar_chart', figure=overall_messages))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([ dbc.Col(dcc.Graph(id='fig2_bar_chart', figure=messages_per_member_continuous))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='fig_icons', figure = icons)),
                        dbc.Col(dcc.Graph(id='fig_subjects', figure=subjects))
                    ],style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='fig_exits', figure=exits)),
                        dbc.Col(dcc.Graph(id='fig_cloud', figure=clouds))
                    ],style={'width': '100%', 'min-width': '1000px','margin':'auto'})
                    ]),

                dcc.Tab(label='Year Based Analysis', children=[
                    html.Br(),
                    html.H1(id = "html tab two", children = ["Analysis for the year {}".format(obj.ls_years[0])]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    html.Br(),
                    dcc.Dropdown(id="dropdown", options=obj.ls_options, searchable=False, placeholder="Select Year"
                                ,style={'width':'400px', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    html.Div(id = "first_loaded_div",children = [
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='fig4_line_chart', figure=line_chart_mess_per_person_based_year))],
                                style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='fig3_bar_chart', figure=bar_chart_mess_per_person)),
                            dbc.Col(dcc.Graph(id='fig5_bar_chart', figure = fig_day))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='fig6_bar_chart', figure = fig_hour)),
                            dbc.Col(dcc.Graph(id='fig7_bar_chart', figure = fig_date))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'})
                        ]),
                    html.Div(id = 'sample') ]), 

                dcc.Tab(label='Group Chat Analysis', children=[
                    html.Br(),            
                    html.H1("Analysis on text, media and links shared in your group",
                            style={'width': '90%','min-width': '1000px','textAlign': 'center', 'color' : '#837c7c','margin':'auto'}),            

                    html.Br(),   

                    dcc.Dropdown(id="dropdown_sort", options=obj.ls_sort_options, searchable=False, placeholder="Sort columns by"
                              ,multi=True  ,style={'width':'95%', 'min-width': '1000px','margin':'auto'}),       
                    html.Br(), 

                    html.Div(id = "replace", children = [
                    dbc.Table.from_dataframe(df = obj.chat_analysis_data_frame, striped = True ,header = True,
                    style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                                 , className = 'table-dark table-sm')]),

                    html.Br(), 

                    dbc.Row([
                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                            dbc.Table.from_dataframe(obj.dic_links_authors_df['Overall'].head(5).reset_index(), striped = True ,
                            style = {'height':'77.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]),

                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                            dbc.Table.from_dataframe(obj.dic_author_emoji_df['Overall'].head(5).reset_index(), striped = True ,
                            style = {'height':'77.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]),

                        dbc.Col([
                            dcc.Dropdown(id="dropdown_word_len", options=obj.ls_wordlen_options, searchable=False,
                                         placeholder="Select word length",style={'width' : '200px'}),
                            html.Br(),
                            html.Div(id = "col_overall_word_len", children = [
                            dbc.Table.from_dataframe(obj.dic_df_author_word_count['Overall'].head(5).reset_index(),
                                striped = True, style = {'height':'80.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]) ])

                    ],style={'width':'92%','margin':'auto', 'min-width': '1000px'}),

                    dcc.Dropdown(id="dropdown_author", options=obj.ls_author_options, searchable=False, placeholder="Select group member",
                        value = sorted(obj.top_authors.tolist())[0] ,style={'width':'94.5%', 'min-width': '1000px','margin':'auto'}),   

                    html.Br(),  

                    html.Div(id="author_details",style = {'font-size': '13px','textAlign': 'center','width':'89%', 'min-width': '1000px','margin':'auto'})
            ]),
            ], colors={  "border": "white","primary": "gold","background": "cornsilk"}, style={'width': '100%', 'min-width': '1000px','margin':'auto'})

            return tab
        else:
            tab = dcc.Tabs([
                dcc.Tab(label='OverAll Group Analysis', children=[
                    html.Br(),
                    html.H1("OverAll Group Messages Analysis"
                            ,style={'width':'100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    dbc.Row([ dbc.Col(dcc.Graph(id='fig1_bar_chart', figure=overall_messages))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([ dbc.Col(dcc.Graph(id='fig2_bar_chart', figure=messages_per_member_continuous))]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='fig_icons', figure = icons)),
                        dbc.Col(dcc.Graph(id='fig_subjects', figure=subjects))
                    ],style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    dbc.Row([
                        dbc.Col(dcc.Graph(id='fig_exits', figure=exits)),
                        dbc.Col(dcc.Graph(id='fig_cloud', figure=clouds))
                    ],style={'width': '100%', 'min-width': '1000px','margin':'auto'})
                    ]),

                dcc.Tab(label='Year Based Analysis', children=[
                    html.Br(),
                    html.H1(id = "html tab two", children = ["Analysis for the year {}".format(obj.ls_years[0])]
                            ,style={'width': '100%', 'min-width': '1000px','margin':'auto','textAlign': 'center', 'color' : '#837c7c'}),
                    html.Br(),
                    dcc.Dropdown(id="dropdown", options=obj.ls_options, searchable=False, placeholder="Select Year"
                                ,style={'width':'400px', 'min-width': '1000px','margin':'auto'}),
                    html.Br(),
                    html.Div(id = "first_loaded_div",children = [
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='fig4_line_chart', figure=line_chart_mess_per_person_based_year))],
                                style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='fig3_bar_chart', figure=bar_chart_mess_per_person)),
                            dbc.Col(dcc.Graph(id='fig5_bar_chart', figure = fig_day))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='fig6_bar_chart', figure = fig_hour)),
                            dbc.Col(dcc.Graph(id='fig7_bar_chart', figure = fig_date))]
                                ,style={'width': '100%', 'min-width': '1000px','margin':'auto'})
                        ]),
                    html.Div(id = 'sample') ]), 

                dcc.Tab(label='Group Chat Analysis', children=[
                    html.Br(),            
                    html.H1("Analysis on text, media and links shared in your group",
                            style={'width': '90%','min-width': '1000px','textAlign': 'center', 'color' : '#837c7c','margin':'auto'}),            

                    html.Br(),   

                    dcc.Dropdown(id="dropdown_sort", options=obj.ls_sort_options, searchable=False, placeholder="Sort columns by"
                              ,multi=True  ,style={'width':'95%', 'min-width': '1000px','margin':'auto'}),       
                    html.Br(), 

                    html.Div(id = "replace", children = [
                    dbc.Table.from_dataframe(df = obj.chat_analysis_data_frame, striped = True ,header = True,
                    style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                                 , className = 'table-dark table-sm')]),

                    html.Br(), 

                    dbc.Row([
                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                           html.H1("No Links Available for your group") ]),

                        dbc.Col([
                            html.Div(" ", style = {'height':'38px'}),   html.Br(),
                            dbc.Table.from_dataframe(obj.dic_author_emoji_df['Overall'].head(5).reset_index(), striped = True ,
                            style = {'height':'77.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]),

                        dbc.Col([
                            dcc.Dropdown(id="dropdown_word_len", options=obj.ls_wordlen_options, searchable=False,
                                         placeholder="Select word length",style={'width' : '200px'}),
                            html.Br(),
                            html.Div(id = "col_overall_word_len", children = [
                            dbc.Table.from_dataframe(obj.dic_df_author_word_count['Overall'].head(5).reset_index(),
                                striped = True, style = {'height':'80.5%','font-size': '13px','textAlign': 'center'}, className = 'table-dark' ,  header = True) ]) ])

                    ],style={'width':'92%','margin':'auto', 'min-width': '1000px'}),

                    dcc.Dropdown(id="dropdown_author", options=obj.ls_author_options, searchable=False, placeholder="Select group member",
                        value = sorted(obj.top_authors.tolist())[0] ,style={'width':'94.5%', 'min-width': '1000px','margin':'auto'}),   

                    html.Br(),  

                    html.Div(id="author_details",style = {'font-size': '13px','textAlign': 'center','width':'89%', 'min-width': '1000px','margin':'auto'})
            ]),
            ], colors={  "border": "white","primary": "gold","background": "cornsilk"}, style={'width': '100%', 'min-width': '1000px','margin':'auto'})

            return tab
        
    
@app.callback([Output(component_id='html tab two', component_property='children'),
        Output(component_id='first_loaded_div', component_property='hidden'),
        Output(component_id='sample', component_property='children')] ,
        [Input(component_id="dropdown",component_property="value")]
    )
def update_on_change( value):
    if value is None:
        raise PreventUpdate
    else:
        progress = obj.ls_years_df[obj.ls_years.index(value)].copy()
        overall_messages = fig_overall_messages(obj.hover_label_style, obj.axis_style, obj.margin_style, obj.button_style, obj.ls_years_df, obj.ls_years)
        bar_chart_mess_per_person = fig_bar_chart_mess_per_person(progress ,value, obj.hover_label_style, obj.axis_style, obj.margin_style)
        line_chart_mess_per_person_based_year = fig_line_chart_mess_per_person_based_year(progress ,value, obj.hover_label_style, obj.axis_style, obj.margin_style, obj.button_style)
        fig_day, fig_hour, fig_date =fig_active_day_hour_date(progress ,value, obj.hover_label_style, obj.axis_style, obj.margin_style)
        chart = [
                dbc.Row([
                        dbc.Col(dcc.Graph(id='fig4_line_chart', figure=line_chart_mess_per_person_based_year))],
                            style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                dbc.Row([
                    dbc.Col(dcc.Graph(id='fig3_bar_chart', figure=bar_chart_mess_per_person)),
                    dbc.Col(dcc.Graph(id='fig5_bar_chart', figure = fig_day))]
                        ,style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                dbc.Row([
                    dbc.Col(dcc.Graph(id='fig6_bar_chart', figure = fig_hour)),
                    dbc.Col(dcc.Graph(id='fig7_bar_chart', figure = fig_date))]
                        ,style={'width': '100%', 'min-width': '1000px','margin':'auto'})
                ]
        return  "Analysis for the year {}".format(value), "hidden" ,chart

@app.callback(Output(component_id='replace', component_property='children') ,
        [Input(component_id="dropdown_sort",component_property="value")]
    )
def update_on_sort( value):
    if value is None:
        raise PreventUpdate
    else:
        ls_values = ['Group Member'] + value
        if(len(ls_values)>1):
            table = dbc.Table.from_dataframe(df = obj.chat_analysis_data_frame.sort_values(value, ascending = False)[ls_values], striped = True ,
                style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                             , className = 'table-dark table-sm' ,  header = True)
        else:
            table = dbc.Table.from_dataframe(df = obj.chat_analysis_data_frame, striped = True ,
                style = {'textAlign': 'center','width':'90%','min-width': '1000px','margin':'auto'}
                             , className = 'table-dark table-sm' ,  header = True)
        return  table

@app.callback(Output(component_id='col_overall_word_len', component_property='children') ,
        [Input(component_id="dropdown_word_len",component_property="value")]
    )
def update_on_overall_wordlen(value):
    if value is None:
        table = dbc.Table.from_dataframe(obj.dic_df_author_word_count['Overall'].head(5).reset_index(), striped = True,
                style = {'textAlign': 'center','font-size': '13px'}, className = 'table-dark' ,  header = True)
        return  table
    else:
        df = obj.dic_df_author_word_count['Overall'].reset_index().apply(lambda x : [x[0],x[1]] if len(x[0])==value else None, axis=1,result_type='broadcast').dropna()
        table = dbc.Table.from_dataframe(df.head(5), striped = True ,
                style = {'textAlign': 'center','font-size': '13px'}, className = 'table-dark' ,  header = True)
        return  table
       
@app.callback(Output(component_id='author_details', component_property='children') ,
        [Input(component_id="dropdown_author",component_property="value")]
    )            
def update_on_author( value):
    if value is None:
        raise PreventUpdate
    else:
        links = list(obj.dic_links_authors_df.keys())[:-1]
        emojis = list(obj.dic_author_emoji_df.keys())[:-1]
        words = list(obj.dic_df_author_word_count.keys())[:-1]
        all_posibilities = set(emojis).intersection(words).intersection(links)
        links_emojis = set(links).intersection(emojis)
        links_words = set(links).intersection(words)
        words_emojis = set(emojis).intersection(words)
        if value in all_posibilities:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px', } , className = 'table-dark' ,  header = True) ]),
                dbc.Col(width = '34%', children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    dcc.Dropdown(id="dropdown_wordlen_author", options=obj.ls_wordlen_options, searchable=False,
                                value = 3,  placeholder="Select word length", style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "table_author_word_len",df = obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])
            return table_author
        elif value in links_emojis:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col(width = '34%', children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    html.H1("No words available for {}".format(value)) ])
                ])
            return table_author
        elif value in links_words:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),                    
                    dbc.Table.from_dataframe(obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Emojis Available for {}".format(value)) ]),
                dbc.Col([
                    dcc.Dropdown(id="dropdown_wordlen_author", options=obj.ls_wordlen_options, searchable=False,
                                 value = 3, placeholder="Select word length", style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "table_author_word_len",df = obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])            
            return table_author
        elif value in words_emojis:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Links Available for {}".format(value)) ]),
                dbc.Col(width = '34%',children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    dcc.Dropdown(id="dropdown_wordlen_author", options=obj.ls_wordlen_options, searchable=False,
                                 value = 3, placeholder="Select word length" ,style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "table_author_word_len",df = obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])            
            return table_author
        elif value in emojis:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Links Available for {}".format(value)) ]),
                dbc.Col(width = '34%',children = [
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(obj.dic_author_emoji_df[value].head(5).reset_index(), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Words Available for {}".format(value)) ])
                ])            
            return table_author
        elif value in links:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    dbc.Table.from_dataframe(obj.dic_links_authors_df[value].head(5), striped = True 
                    , style = {'height':'80%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Emojis Available for {}".format(value)) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Words Available for {}".format(value)) ])
                ])            
            return table_author
        elif value in words:
            table_author = dbc.Row([
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Links Available for {}".format(value)) ]),
                dbc.Col([
                    html.Div(" ", style = {'height':'38px'}),   html.Br(),
                    html.H1("No Emojis Available for {}".format(value)) ]),
                dbc.Col([
                    dcc.Dropdown(id="dropdown_wordlen_author", options=obj.ls_wordlen_options, searchable=False,
                                 value = 3, placeholder="Select word length" ,style={'width' : '200px'}),
                    html.Br(),
                    dbc.Table.from_dataframe(id = "table_author_word_len",df = obj.dic_df_author_word_count[value].head(5), striped = True 
                    , style = {'height':'80.5%','width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True) ])
                ])            
            return table_author
        
        else:
            return html.H1("No Details Available for {}".format(value))
        
@app.callback(Output(component_id='table_author_word_len', component_property='children') ,
        [Input(component_id="dropdown_wordlen_author",component_property="value"),
        Input(component_id="dropdown_author",component_property="value")]
    )
def update_on_author_wordlen(value,author):
    if value is None:
        raise PreventUpdate
    else:
        df = obj.dic_df_author_word_count[author].apply(lambda x : [x[0],x[1]] if len(x[0])==value else None, axis=1,result_type='broadcast').dropna()
        if len(df)>0:
            table = dbc.Table.from_dataframe(df.head(5), striped = True 
                  , style = {'width':'100%','font-size': '13px'} , className = 'table-dark' ,  header = True)
            return  table
        else:
            return html.H1("No Words available for {} of length {}".format(author,value))

In [25]:
@app.callback(Output('personal', 'children'),
              [Input('personal_submit', 'n_clicks')])
def displayClick(btn1):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'personal_submit' in changed_id:
        return html.Div(id = "group", children = [
            html.Div([
            html.Br(),
            html.H1('WhatsApp Group Chat Analysis'
                ,style={'height': '20px', 'color': 'white','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Button('Switch to Personal Chat', id='group_submit', n_clicks=0,
            style = {'margin-left' : '80%', 'color': 'white','background-color': 'white', 'borderRadius': '5px',
                     'height': '40px', 'border': 'none', 'color': 'black', 'text-align': 'center',
                     'text-decoration': 'none', 'display': 'inline-block','font-size': '16px '}),
            html.Br(),html.Br(),
            dcc.Upload(id='upload',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select the Group Chat text file')
                ],style={'background-color': 'white','min-width': '1000px','width': '20%','margin':'auto','height': '60px','lineHeight': '60px','borderWidth': '1px','borderStyle': 'dashed','borderRadius': '5px'}
                )),
            html.Br(),html.Br(),
                ],style={'min-width': '1000px','textAlign': 'center','margin': 'auto', 'background-color':'#5d5e5e'}),    
            html.Div(id='main tab',children =[
                dcc.Tabs([
                dcc.Tab( children=[
                    html.H1("See how your groups texting behaviour has changed over time",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
                    html.Br(),html.Br(),
                    dbc.Row(children = [
                        dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/Tab1_1.png', width = "95%")),
                        dbc.Col(
                            html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/Tab1_2.png', width = "95%")
                        ),
                    ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
                    html.Br(),html.Br(),

                    html.H1("See who are the most active group members and other group activities",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
                    html.Br(),html.Br(),
                    dbc.Row(children = [
                        dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/Tab2_3.png', width = "95%")),
                        dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/Tab2_2.png', width = "95%")),
                    ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
                    html.Br(),html.Br(),

                    html.H1("See analysis on top words, emojis and links used in your group",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
                    html.Br(),html.Br(),
                    dbc.Row(children = [
                        dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/Tab3_5.PNG',height = "80%",width = "80%")),
                        dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/Tab1_6.PNG', height = "80%",width = "80%"), style = {'margin-left':'20px'}),
                        dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/Tab3_2.png', height = "80%",width = "80%"), style = {'margin-left':'20px'}),
                    ], style = {'min-width': '1000px','width': '95%','margin':'auto'}),
                    html.H1("& much much more......",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
                    html.Br(),

                ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),

            ],colors={  "border": "white","primary": "gold","background": "cornsilk"}, style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                                             ]),
        ])
    else:
        raise PreventUpdate

@app.callback(Output('group', 'children'),
              [Input('group_submit', 'n_clicks')])
def displayClick(btn1):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'group_submit' in changed_id:
        return html.Div(id= "personal", children = [
    
    html.Div([
    html.Br(),
    html.H1('WhatsApp Personal Chat Analysis'
    ,style={'height': '20px', 'color': 'white','min-width': '1000px','width': '20%','margin':'auto'}),
    html.Button('Switch to Group Chat', id='personal_submit', n_clicks=0,
            style = {'margin-left' : '80%', 'color': 'white','background-color': 'white', 'borderRadius': '5px',
                     'height': '40px', 'border': 'none', 'color': 'black', 'text-align': 'center',
                     'text-decoration': 'none', 'display': 'inline-block','font-size': '16px '}),
    html.Br(),html.Br(),
    dcc.Upload(id='personal_upload',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select the Personal Chat text file')
        ],style={'background-color': 'white','min-width': '1000px','width': '20%','margin':'auto','height': '60px','lineHeight': '60px','borderWidth': '1px','borderStyle': 'dashed','borderRadius': '5px'}
        )),
    html.Br(),html.Br(),
        ],style={'min-width': '1000px','textAlign': 'center','margin': 'auto', 'background-color':'#5d5e5e'}),    
    html.Div(id='personal_main tab',children =[
        dcc.Tabs([
        dcc.Tab( children=[
            html.H1("See how your texting behaviour has changed over time",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),html.Br(),
            dbc.Row(children = [
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_overall_messages.png', width = "95%")),
                dbc.Col(
                    html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_continuous.png', width = "95%")
                ),
            ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
            html.Br(),html.Br(),
            
            html.H1("See who is the most active during various time periods",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),html.Br(),
            dbc.Row(children = [
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_messages_pie.png', width = "95%")),
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_week_activity.png', width = "95%")),
            ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
            html.Br(),html.Br(),
            
            html.H1("See analysis on top words, emojis and links used",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),html.Br(),
            dbc.Row(children = [
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_emojis.png',height = "80%",width = "95%")),
                dbc.Col(html.Img(src = 'https://raw.githubusercontent.com/mazeenmuhammed/Assets/master/assets/personal_word_count.png', height = "80%",width = "100%"))
            ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
            html.H1("& much much more......",style={'font-size': '25px','textAlign': 'center','height': '20px','min-width': '1000px','width': '20%','margin':'auto'}),
            html.Br(),
            
        ], style = {'min-width': '1000px','width': '100%','margin':'auto'}),
    
    ],colors={  "border": "white","primary": "gold","background": "cornsilk"}, style={'width': '100%', 'min-width': '1000px','margin':'auto'}),
                                     ]),
])
    else:
        raise PreventUpdate

In [26]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2020 12:01:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2020 12:01:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2020 12:01:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2020 12:01:15] "GET /_favicon.ico?v=1.14.0 HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2020 12:01:15] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [15/Nov/2020 12:01:15] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [15/Nov/2020 12:01:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2020 12:01:23] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [15/Nov/2020 12:01:23] "POST /_dash-update-component HTTP/1.1" 204 -
